# Single Notebook With All the Code to Go From List of Science Job Titles to geoJSON of Companies advertising those jobs on a map

------------------------------------------------

# General Outline
### get list of science related jobs
    * Clean up a list of science related job titles I modified from wikipedia in excel, save as csv
### get company names that are hiring for those science related job titles in Houston, TX by scraping job search site(s)
    * use beautiful soup, a python library to gather information from websites programtically and return list of companies
    * Clean up duplication and companies that shouldn't be included
### get addresses and lat longs associated with each company name in json form
    * use google places API to search for company name and a Houston, TX location, returning json of location and company information
    * clean returned list of json data such that it is limited to a lat long box of the greater houston area
    * clean up duplicates and are false positives useing pandas and other python libraries to 
### convert results to geoJSON. Add-in additional data dimensions, such as job title used in seach, as geoJSON properties
    * use geoJSON and JSON python libraries to convert previous json into points, and then features, and then a single feature collection
### get geoJSONs from other sources that contain science location data on things like schools and hospitals
    * use .... a http get call to access the geoJSON files already created by the city of Houston.
### add geoJSONs to map. Use color, size, and shape to represent different geoJSON properties. Tell a story
    * use mapboxGL.js, leaflet.js, and basic html/CSS/JS to create maps

### importing needed python modules

In [61]:
#### Imports for parts #1 & #2
from bs4 import BeautifulSoup # For HTML parsing
from urllib.request import urlopen
import re # Regular expressions
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
%matplotlib inline
import json
import csv

##### Import modules for google API and geoJSON parts. Only duplicate is json
import requests
import configparser
import json
import geojson
from geojson import Point, Feature, FeatureCollection
# https://github.com/frewsxcv/python-geojson
import time
import pandas as pd

===================================================================================

# 1. Load list of job titles

------------------------------------------------------------------------------

In [40]:
with open('../Data/science_jobs_v2.csv', 'rt') as f:
    reader = csv.reader(f)
    science_jobs_list_withInfo = list(reader)

job_title_list = []
for each in science_jobs_list_withInfo:
    each = each[0]
    #print('each',each)
    job_title_list.append(each)

# take out the header 'Job_Title'
job_title_list.pop(0)

print(job_title_list)

['Bioanalytical Scientist', 'Biochemist', 'Biochemistry', 'Bioinformatics Research Scientist', 'Biological Engineer', 'Biologist', 'biology  ', 'biomechanical', 'Biomedical scientist', 'biophysics', 'Cell Line', 'Clinical Data', 'Clinical Data Research', 'Clinical Pharmacology Professor', 'Clinical Pharmacy Assistant', 'Clinical Research Associate', 'Drug Evaluator', 'Drug Regulatory Affairs Manager', 'Environmental Health Scientist', 'Genetic Counselor', 'Health Research', 'Health Technology', 'Healthcare science', 'Hospital Research Assistant', 'Immunology Scientist', 'Life Science', 'Medical', 'Medical Center', 'medical Laboratory', 'Medical laboratory scientist', 'Medical Physics Researcher', 'Medical Research Assistant', 'Medical Research Technician', 'Medical Scientist', 'Microbiologist', 'Molecular Biologist', 'Molecular physics', 'Neuroscientist', 'Nuclear', 'Oncology Researcher', 'Pathologist', 'Pharmaceutical Assistant', 'Pharmaceutical Research Analyst', 'Pharmaceutical Rese

In [262]:
short_job_title_list = ['Chemist', 'Conservation', 'Forensic Chemist', 'Herpetologist', 'Laboratory', 'meteorologist ', 'Naturalist', 'Oceanographer', 'oceanography', 'optomitrist']

In [263]:
short_job_title_list_old1stHalf = ['Biochemistry', 'Bioinformatics', 'Biologist', 'biomechanical', 'Biomedical', 'biophysics', 'Cell+Line', 'Clinical+Data', 'Pharmacology', 'Pharmacy', 'Drug Evaluator', 'Environmental Health', 'Genetic Counselor', 'Healthcare', 'Hospital', 'Immunology', 'Life Science', 'Medical', 'Medical Center', 'medical Laboratory', 'Medical Physics', 'Medical', 'Microbiologist', 'Neuroscientist', 'Nuclear', 'Oncology', 'Pathologist', 'Pharmaceutical', 'Photochemistry', 'Public Health Specialist', 'radiobiology', 'Radiochemistry', 'Toxicologist', 'Agricultural', 'Archaeologist', 'Associate Professor', 'Botanist', 'botany', 'Chemical', 'Chemist', 'Conservation', 'Forensic Chemist', 'Herpetologist', 'Laboratory', 'meteorologist ', 'Naturalist', 'Oceanographer', 'oceanography', 'optomitrist']

In [264]:
short_job_title_list_2ndHalf = ['Forensic Scientist','Physical Scientist','Psychologist', 'Research Fellow', 'Science teacher', 'Scientist', 'social science', 'Solid-state', 'STEM', 'Stem Cell', 'Stereochemistry', 'Volcanologist', 'volcanology', 'wind power', 'wind turbine ', 'zoo', 'Zoologist', 'zoology ', 'chemistry', 'Crystallography', 'Earth Science', 'Ecologist', 'Ecology', 'Environmental', 'Exploration', 'fluid dynamics', 'geochemist', 'Geochemistry', 'Geographer', 'Geologist', 'geology ', 'geophysicist', 'GiS ', 'Groundwater Technician', 'hydrology', 'Inorganic', 'lab technician', 'Materials science', 'micropaleontologist', 'operations geologist', 'paleoclimatology', 'paleoecology', 'Paleontologist', 'Petroleum geologist', 'petrophysicist', 'playnology', 'reservoir', 'modeler', 'rock Laboratory', 'sedimentologist', 'stratigraphy', 'subsea', 'aeronautics', 'Aerospace', 'astrobiology', 'Astrochemistry', 'astromaterials', 'Astronaut', 'Astronomer', 'astronomy', 'astrophysics', 'Food chemistry', 'heliophysics', 'International Space Station', 'Johnson Space Center', 'lunar', 'materials', 'NASA', 'Physicist', 'physics', 'Planetary', 'propulsion engineer', 'Satellite', 'Space science', 'telemetry']

In [277]:
test_short_job_title_list = ['Chemist', 'Conservation', 'Herpetologist', 'Laboratory', 'meteorologist ', 'Naturalist']

In [265]:
len(short_job_title_list)

10

In [266]:
# replace some of the characters so the search is a bit better:
def cleanCompanies(company_array):
    new_array = []
    for each in company_array:
        #print(each, "and type is ",type(each))
        print(each.find("-",0))
        print(type(each.find("-",0)),"type of each.find('-',0")
        #print(type(each.find("%2C",1)))
        if each.find("%2C",1) is not -1:
            each = each.replace("%2C",",")
            print(each,"= each")
        else:
            each = each
        if each.find("%26",1) is not -1:
            each = each.replace("%26","&")
            #print(each,"= each find")
        else:
            each = each
        ##print(each, "again with each")
        if  each.find("-1",1) is not -1:
            each = each.replace("-1","")
        #    print(each,"= each find")
        else:
            each = each
        if  each.find("-2",0) is not -1:
            each = each.replace("-2","")
            print(each,"= each find")
        else: 
            each = each
        if  each.find("-3",0) is not -1:
            each = each.replace("-3","")
        #    print(each,"= each find")
        else:
            each
        if  each.find("-",1) is not -1:
            each = each.replace("-","+")
            print(each,"= each find")
        else:
            each = each
        if  each.find(" ",1) is not -1:
            each = each.replace(" ","+")
            print(each,"= each find")
        else:
            each = each
        print("each at end",each)
        new_array.extend([each])
    print('new_array',new_array)
    return new_array

In [267]:
short_job_title_list = cleanCompanies(short_job_title_list)
short_job_title_list

-1
<class 'int'> type of each.find('-',0
each at end Chemist
-1
<class 'int'> type of each.find('-',0
each at end Conservation
-1
<class 'int'> type of each.find('-',0
Forensic+Chemist = each find
each at end Forensic+Chemist
-1
<class 'int'> type of each.find('-',0
each at end Herpetologist
-1
<class 'int'> type of each.find('-',0
each at end Laboratory
-1
<class 'int'> type of each.find('-',0
meteorologist+ = each find
each at end meteorologist+
-1
<class 'int'> type of each.find('-',0
each at end Naturalist
-1
<class 'int'> type of each.find('-',0
each at end Oceanographer
-1
<class 'int'> type of each.find('-',0
each at end oceanography
-1
<class 'int'> type of each.find('-',0
each at end optomitrist
new_array ['Chemist', 'Conservation', 'Forensic+Chemist', 'Herpetologist', 'Laboratory', 'meteorologist+', 'Naturalist', 'Oceanographer', 'oceanography', 'optomitrist']


['Chemist',
 'Conservation',
 'Forensic+Chemist',
 'Herpetologist',
 'Laboratory',
 'meteorologist+',
 'Naturalist',
 'Oceanographer',
 'oceanography',
 'optomitrist']

In [268]:
len(short_job_title_list)

10

In [269]:
len(new_list)

165

------------------------------------------------------------------------------
# 2. Get company names that are hiring for those science related job titles in Houston, TX by scraping job search site(s)
------------------------------------------------------------------------------


=========================================================================================
## A data exploration using beautiful soup
- based on this: https://jessesw.com/Data-Science-Skills/

### The function 'skills_info' takes three arguments; city, state, and job title and returns a list of companies searching for that job title in the last 30 days in Houston, TX

In [42]:
def skills_info(city="Houston", state="TX",job_title='data+scientist'):
    '''
    This function will take a desired city/state and look for all new job postings
    on Indeed.com. It will crawl all of the job postings and keep track of how many
    use a preset list of typical data science skills. The final percentage for each skill
    is then displayed at the end of the collation. 
        
    Inputs: The location's city and state. These are optional. If no city/state is input, 
    the function will assume a national search (this can take a while!!!).
    Input the city/state as strings, such as skills_info('Chicago', 'IL').
    Use a two letter abbreviation for the state.
    '''
    #print("test1")   
    final_job = job_title # searching for data scientist exact fit("data scientist" on Indeed search)
    
    # Make sure the city specified works properly if it has more than one word (such as San Francisco)  %22&l=
    if city is not None:
        final_city = city.split() 
        final_city = '+'.join(word for word in final_city)
        final_site_list = ['https://www.indeed.com/jobs?q=', final_job, '&l=', final_city,'%2C+', state] # Join all of our strings together so that indeed will search correctly
    else:
        final_site_list = ['https://www.indeed.com/jobs?q="', final_job, '"']

    final_site = ''.join(final_site_list) # Merge the html address together into one string

    # website query today (2017-02-25) is = https://www.indeed.com/jobs?q=data+scientist&l=Houston%2CTX
        
    base_url = 'http://www.indeed.com'
    
    #print("test2")  
    try:
        html = urlopen(final_site).read() # Open up the front page of our search first
    except:
        #print('That city/state combination did not have any jobs. Exiting . . .') # In case the city is invalid
        return
    soup = BeautifulSoup(html,"lxml") # Get the html from the first page
    
    # Now find out how many jobs there were
    
    if type(soup.find(id = 'searchCount')) is None or type(soup.find(id = 'searchCount')) == None:
        c
    
    else:
        variableS = soup.find(id = 'searchCount')
        if isinstance(variableS,type(None)):
            all_company_results = []
            return all_company_results
        else:
            print("soup.find(id = 'searchCount') = ",soup.find(id = 'searchCount'))
            print("soup.find(id = 'searchCount') = ",type(soup.find(id = 'searchCount')))

            num_jobs_area = soup.find(id = 'searchCount').string.encode('utf-8') # Now extract the total number of jobs found
                                                                                # The 'searchCount' object has this
            #print('num_jobs_area before filtering ', num_jobs_area)

            job_numbers = re.findall(b'\d+', num_jobs_area) # Extract the total jobs found from the search result

            #print('job number before filtering ', job_numbers[2])

            job_numbers_decoded = int(job_numbers[2].decode('utf-8'))
            #print('job_numbers_decoded = ',job_numbers_decoded, "type is ",type(job_numbers_decoded))

            #if len(job_numbers[2]) > 3: # Have a total number of jobs greater than 1000
            #    total_num_jobs = (int(job_numbers[2])*1000) + int(job_numbers[3])
            #else:
            #    total_num_jobs = int(job_numbers[2]) 

            total_num_jobs = job_numbers_decoded

            city_title = city
            if city is None:
                city_title = 'Nationwide'

            #print('There were', total_num_jobs, 'jobs found,', city_title) # Display how many jobs were found

            # took out /10 here, which seemed to be assuming always 10 results per page? now 15?
            num_pages = int(float(total_num_jobs/10)) # This will be how we know the number of times we need to iterate over each new
                                              # search result page

            job_descriptions = [] # Store all our descriptions in this list

            # used to be 'xrange' here but I changed it to 'range' because I'm using python3 and xrange is python2 only
            all_company_results = []
            for i in range(1,num_pages+1): # Loop through all of our search result pages
                #print('Getting page', i)

                start_num = str(i*10) # Assign the multiplier of 10 to view the pages we want
                current_page = ''.join([final_site, '&start=', start_num])
                # Now that we can view the correct 10 job returns, start collecting the text samples from each

                html_page = urlopen(current_page).read() # Get the page

                page_obj = BeautifulSoup(html_page,"lxml") # Locate all of the job links
                job_link_area = page_obj.find(id = 'resultsCol') # The center column on the page where the job postings exist
                #print('page_obj',page_obj)  

                #print("link.get('href')",base_url + link.get('href')
                #print("job_link_area.find_all('a')",job_link_area.find_all('a'))

                for a in job_link_area.find_all('a', href=True):
                    #print ("Found the URL:", a['href'])
                    if "/cmp/" in a['href']: 
                        #  if a['href'].find("/cmp/"):
                        #print("a had /cmp/ in it using find = ",a['href'])
                        arrayHrefParts_0 = a['href'].split('/cmp/')
                        #print("aarrayHrefParts_0 = ",arrayHrefParts_0)
                        arrayHrefParts_1 = arrayHrefParts_0[1].split('/')
                        #print("arrayHrefParts_1 = ",arrayHrefParts_1)
                        company = arrayHrefParts_1[0]
                        #print('company = ',company)
                        all_company_results.append(company)
                        #print('all_company_results =',all_company_results)
                    else:
                        continue
                        #print("/cmp/ not found in ,",a['href'])

                #print("[base_url + link.get('href') for link in job_link_area.find_all('a')]",[base_url + link.get('href') for link in job_link_area.find_all('a')])

                #job_URLS = [base_url + link.get('href') for link in job_link_area.find_all('a')] # Get the URLS for the jobs

                #job_URLS = filter(lambda x:'clk' in x, job_URLS) # Now get just the job related URLS


                #         for j in xrange(0,len(job_URLS)):
                #             final_description = text_cleaner(job_URLS[j])
                #             if final_description: # So that we only append when the website was accessed correctly
                #                 job_descriptions.append(final_description)
                #             sleep(1) # So that we don't be jerks. If you have a very fast internet connection you could hit the server a lot! 

                sleep(1) # So that we don't be jerks. If you have a very fast internet connection you could hit the server a lot! 

            for each in range(len(all_company_results)):
                if '.html' in all_company_results[each]:
                    #print('all_company_results[each]',all_company_results[each]," each = ",each)
                    new_name_wo_html = all_company_results[each].split('.html')[0]
                    #print('new_name_wo_html',new_name_wo_html)
                    all_company_results[each] = new_name_wo_html  

            all_company_results = list(set(all_company_results))
            #print ('Done with collecting the job postings!')    
            #print ('all_company_results final = ',all_company_results)
            #print ("There were",len(all_company_results)," companies successfully found.")
            all_company_results_json = []
            for each in all_company_results:
                print("each in all_company_results = ",each)
                print("job_title in all_company_results = ",job_title)
                temp_obj_holder = {"company":"nothing","job_title":"nothing"}
                print("temp_obj_holder = ",temp_obj_holder)
                temp_obj_holder["job_title"] = job_title
                temp_obj_holder["company"] = each
                print("temp_obj_holder['job_title'] = ",temp_obj_holder["job_title"])
                all_company_results_json.append(temp_obj_holder)
            return all_company_results_json


### The function 'runMultiple_skills_info' runs the 'skills_info' function multiple times, one for each job title in the list given to 'runMultiple_skills_info' function

In [43]:
def runMultiple_skills_info(array_of_jobTitles=['puppies']):
    all_results = []
    for each in array_of_jobTitles:
        print('each= ',each)
        #temp_results1 = []
        temp_results1 = skills_info("Houston","TX",each)
        if temp_results1 is None:
            continue
        else:
            print('temp_results is ',temp_results1)
            #all_results.extend(temp_results1) 
            all_results.extend(temp_results1)
    return all_results

###  | run a first test, should get back an array of objects with keys of company and job_title

In [77]:
array2 = ['geologist','geophysicist']


### create a small array for testing purposes

In [78]:
geo_and_geo = runMultiple_skills_info(array_of_jobTitles=array2)

each=  geologist
soup.find(id = 'searchCount') =  <div id="searchCount">Jobs 1 to 10 of 44</div>
soup.find(id = 'searchCount') =  <class 'bs4.element.Tag'>
each in all_company_results =  Rbc
job_title in all_company_results =  geologist
temp_obj_holder =  {'company': 'nothing', 'job_title': 'nothing'}
temp_obj_holder['job_title'] =  geologist
each in all_company_results =  Hess-Corporation
job_title in all_company_results =  geologist
temp_obj_holder =  {'company': 'nothing', 'job_title': 'nothing'}
temp_obj_holder['job_title'] =  geologist
each in all_company_results =  Tolunay--wong-Engineers,-Inc.
job_title in all_company_results =  geologist
temp_obj_holder =  {'company': 'nothing', 'job_title': 'nothing'}
temp_obj_holder['job_title'] =  geologist
each in all_company_results =  Bhp-Billiton
job_title in all_company_results =  geologist
temp_obj_holder =  {'company': 'nothing', 'job_title': 'nothing'}
temp_obj_holder['job_title'] =  geologist
each in all_company_results =  Enrud-Res

In [79]:
geo_and_geo

[{'company': 'Rbc', 'job_title': 'geologist'},
 {'company': 'Hess-Corporation', 'job_title': 'geologist'},
 {'company': 'Tolunay--wong-Engineers,-Inc.', 'job_title': 'geologist'},
 {'company': 'Bhp-Billiton', 'job_title': 'geologist'},
 {'company': 'Enrud-Resources', 'job_title': 'geologist'},
 {'company': 'W&T-Offshore,-Inc.', 'job_title': 'geologist'},
 {'company': 'SAN-Jacinto-College', 'job_title': 'geologist'},
 {'company': 'Bmo-Financial-Group', 'job_title': 'geologist'},
 {'company': 'Jefferies', 'job_title': 'geologist'},
 {'company': 'Fugro', 'job_title': 'geologist'},
 {'company': 'Walker-Elliott', 'job_title': 'geologist'},
 {'company': 'Hilcorp-Energy-Company', 'job_title': 'geologist'},
 {'company': 'Occidental-Petroleum', 'job_title': 'geologist'},
 {'company': 'Murray-Resources', 'job_title': 'geologist'},
 {'company': 'Intera', 'job_title': 'geologist'},
 {'company': 'High-Country-Executive-Search', 'job_title': 'geologist'},
 {'company': 'Weatherford', 'job_title': 'ge

In [276]:
def saveArray_of_JobSearchResults(arrayOfJobsAndSearchTerm = geo_and_geo, FilePath ='../DATA/someFile'):
    toCSV = arrayOfJobsAndSearchTerm
    keys = toCSV[0].keys()
    with open(FilePath, 'w') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(toCSV)
    return arrayOfJobsAndSearchTerm
    

In [91]:
saveArray_of_JobSearchResults(arrayOfJobsAndSearchTerm = geo_and_geo, FilePath ='../DATA/test_jobSearchTermAndCompany_geoAndgeo_v4.csv')

####  | end test

#### Review:

3 functions:
- A. skills_info(city="Houston", state="TX",job_title='data+scientist')
- B. runMultiple_skills_info(array_of_jobTitles=['puppies'])
- C. saveArray_of_JobSearchResults(arrayOfJobsAndSearchTerm = geo_and_geo, FilePath ='../DATA/someFile')

A does the web scraping. B does web scraping searching for each of the items in the input array and combines the resuls. C saves the results as a csv file with two columns. One column header is Job Title used in search. The other column is company name looking for that job title in Houston Texas. 

------------------------------------------------------------------------------
# 3. Get addresses and lat longs associated with each company name in json form
------------------------------------------------------------------------------


### Modules imported at top of this notebook

### using <a href="https://docs.python.org/3/library/configparser.html">configparser</a> to handle config file 

In [82]:
config = configparser.ConfigParser()

In [83]:
configFilePath = '../config.conf'
# configparser.ConfigParser().read(configFilePath)
config.read(configFilePath)

['../config.conf']

### next block sets a variety of useful variables used in the google places API

In [84]:
#### sets key as variable that contains the google-places-api key
key = config['google-places-api']['key1']
#### creates a string used as the prefix to the key in the url used in calling the API service
key_pre = '&key='
#### the base url in the google places API get call
base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?query='
#### the base url when calling a next page, in other words, when results are more than 20
next_page_base_url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?'

### load into memory the csv previously created with the company names and job titles that go with them

In [92]:
# using pandas as pd. Pandas should already be imported
df=pd.read_csv('../DATA/test_jobSearchTermAndCompany_geoAndgeo_v4.csv', sep=',',header=None)
company_array = df.values

len(company_array)

40

In [93]:
company_array

array([['company', 'job_title'],
       ['Rbc', 'geologist'],
       ['Hess-Corporation', 'geologist'],
       ['Tolunay--wong-Engineers,-Inc.', 'geologist'],
       ['Bhp-Billiton', 'geologist'],
       ['Enrud-Resources', 'geologist'],
       ['W&T-Offshore,-Inc.', 'geologist'],
       ['SAN-Jacinto-College', 'geologist'],
       ['Bmo-Financial-Group', 'geologist'],
       ['Jefferies', 'geologist'],
       ['Fugro', 'geologist'],
       ['Walker-Elliott', 'geologist'],
       ['Hilcorp-Energy-Company', 'geologist'],
       ['Occidental-Petroleum', 'geologist'],
       ['Murray-Resources', 'geologist'],
       ['Intera', 'geologist'],
       ['High-Country-Executive-Search', 'geologist'],
       ['Weatherford', 'geologist'],
       ['University-of-Houston--downtown', 'geologist'],
       ['Jefferies-&-Company,-Inc.', 'geologist'],
       ['Oasis-Petroleum', 'geologist'],
       ['Hart-Energy', 'geologist'],
       ['Sinopec-Tech-Houston', 'geologist'],
       ['Citi', 'geologist'],


In [142]:
# replace some of the characters so the search is a bit better:
def cleanCompanies(company_array):
    company_array = company_array[1:]
    new_array = []
    for each in company_array:
        job_title = each[1]
        each = each[0]
        #print(each, "and type is ",type(each))
        print(each.find("-",0))
        print(type(each.find("-",0)),"type of each.find('-',0")
        #print(type(each.find("%2C",1)))
        if each.find("%2C",1) is not -1:
            each = each.replace("%2C",",")
            print(each,"= each")
        else:
            each = each
        if each.find("%26",1) is not -1:
            each = each.replace("%26","&")
            #print(each,"= each find")
        else:
            each = each
        ##print(each, "again with each")
        if  each.find("-1",1) is not -1:
            each = each.replace("-1","")
        #    print(each,"= each find")
        else:
            each = each
        if  each.find("-2",0) is not -1:
            each = each.replace("-2","")
            print(each,"= each find")
        else: 
            each = each
        if  each.find("-3",0) is not -1:
            each = each.replace("-3","")
        #    print(each,"= each find")
        else:
            each
        if  each.find("-",1) is not -1:
            each = each.replace("-","+")
            print(each,"= each find")
        else:
            each = each
        print("each at end",each)
        new_array.extend([[each,job_title]])
    print('new_array',new_array)
    return new_array

In [151]:
company_array_formatted = cleanCompanies(company_array)
company_array_formatted

-1
<class 'int'> type of each.find('-',0
each at end Rbc
4
<class 'int'> type of each.find('-',0
Hess+Corporation = each find
each at end Hess+Corporation
7
<class 'int'> type of each.find('-',0
Tolunay++wong+Engineers,+Inc. = each find
each at end Tolunay++wong+Engineers,+Inc.
3
<class 'int'> type of each.find('-',0
Bhp+Billiton = each find
each at end Bhp+Billiton
5
<class 'int'> type of each.find('-',0
Enrud+Resources = each find
each at end Enrud+Resources
3
<class 'int'> type of each.find('-',0
W&T+Offshore,+Inc. = each find
each at end W&T+Offshore,+Inc.
3
<class 'int'> type of each.find('-',0
SAN+Jacinto+College = each find
each at end SAN+Jacinto+College
3
<class 'int'> type of each.find('-',0
Bmo+Financial+Group = each find
each at end Bmo+Financial+Group
-1
<class 'int'> type of each.find('-',0
each at end Jefferies
-1
<class 'int'> type of each.find('-',0
each at end Fugro
6
<class 'int'> type of each.find('-',0
Walker+Elliott = each find
each at end Walker+Elliott
7
<class 

[['Rbc', 'geologist'],
 ['Hess+Corporation', 'geologist'],
 ['Tolunay++wong+Engineers,+Inc.', 'geologist'],
 ['Bhp+Billiton', 'geologist'],
 ['Enrud+Resources', 'geologist'],
 ['W&T+Offshore,+Inc.', 'geologist'],
 ['SAN+Jacinto+College', 'geologist'],
 ['Bmo+Financial+Group', 'geologist'],
 ['Jefferies', 'geologist'],
 ['Fugro', 'geologist'],
 ['Walker+Elliott', 'geologist'],
 ['Hilcorp+Energy+Company', 'geologist'],
 ['Occidental+Petroleum', 'geologist'],
 ['Murray+Resources', 'geologist'],
 ['Intera', 'geologist'],
 ['High+Country+Executive+Search', 'geologist'],
 ['Weatherford', 'geologist'],
 ['University+of+Houston++downtown', 'geologist'],
 ['Jefferies+&+Company,+Inc.', 'geologist'],
 ['Oasis+Petroleum', 'geologist'],
 ['Hart+Energy', 'geologist'],
 ['Sinopec+Tech+Houston', 'geologist'],
 ['Citi', 'geologist'],
 ['Confidential+++Oil+&+Gas+Company', 'geologist'],
 ['Marathon+Oil', 'geologist'],
 ['New+Tech+Global', 'geologist'],
 ['Lonestar+College+System', 'geologist'],
 ['Geotra

### All the functions that load and clean the company + job title list from csv into memory

In [295]:
def load_listCompanyJobtitle(path):
    df=pd.read_csv(path,header=None)
    company_array = df.values
    company_array_formatted = cleanCompanies(company_array)

### Next few functions run the company names through the google places API

##### Below is the main function that calls the Google Places API to get the initial results

In [279]:
# function that takes a datset that is 
def callGooglePlacesAPI(search_term,nameOfSearch,job_title):
    print('job_title put into googleAPI call is =',job_title)
    dataset_I = requests.get(base_url+search_term+key_pre+key)
    next_page_result = []
    try:
        next_page_token = dataset_I.json()['next_page_token']
        next_page_result = callNextPageResults(nameOfSearch,next_page_token,job_title)
    except: 
        pass
    #####if 'next_page_token' in dataset_I:
    #next_page_token = dataset_I.json()['next_page_token']
    #print("next_page_token",next_page_token)
    #callNextPageResults(nameOfSearch,next_page_token)
    #####next_page_results = makeFeatureCollectionsFromPlaces(dataset_I['next_page_token'],nameOfSearch).json()['results']
    ##### #else: 
    ####    next_page = "none needed"
    dataset = dataset_I.json()['results']
    #if 'next_page_token' in dataset_I:
    #    dataset.extend(next_page_results)
    #    print("dataset up top",dataset)
    #else:
    #    next_page = "none needed"
    array_of_features = []
    for each in dataset:
        photos = []
        # centerpoint for coordinates
        #print(each)
        lat = each['geometry']['location']['lat']
        long = each['geometry']['location']['lng']
        # properties
        address = each['formatted_address']
        id = each['id']
        name = each['name']
        # photos is an array
        try:
            photos = each['photos']
        except:
            photos = []
            #print("photos , attribute error but kept going")
        place_id = each['place_id']
        try:
            rating = each['rating']
        except:
            rating = 'NA'
            #print("rating , attribute error but kept going")
        reference = each['reference']
        # types is an array
        types = each['types']
        testPoint = Point((long, lat))
        test2_geoJSOn = Feature(geometry=testPoint, properties={"job_title":job_title,"name": name,"address":address,"id":id,"photos":photos,"place_id":place_id,"rating":rating,"reference":reference,"types":types})
        array_of_features.extend([test2_geoJSOn])
#     new_FeatureCollection = FeatureCollection(array_of_features) 
#     with open('../DATA/'+nameOfSearch+'_test.geojson', 'w') as f:
#         json.dump(new_FeatureCollection, f)
#     return new_FeatureCollection
    print("array_of_features = ",array_of_features," and next_page_result = ",next_page_result)
    print("types: for "+search_term+" type(next_page_result)= ",type(next_page_result)," and type(array_of_features)=  ",type(array_of_features))
    if next_page_result is None or next_page_result==[]:
        fakeVariable = 3
        #print("no second page")
    else:
        #print("next_page_result passed none test and is ",next_page_result)
        array_of_features.extend(next_page_result)
        #print("array_of_features after extension = ",array_of_features)
    #print("2nd statement of array_of_features = ",array_of_features)
    return array_of_features

##### Below is a secondary function that calls the Google Places API "next page" to get any results further additional to the first 40. 

In [280]:
def callNextPageResults(nameOfSearch,keyForNextPage,job_title):
    time.sleep(5)
    #print('next_page_base_url+"&pagetoken="+keyForNextPage+key_pre+key =',next_page_base_url+"pagetoken="+keyForNextPage+key_pre+key)
    dataset_I = requests.get(next_page_base_url+"pagetoken="+keyForNextPage+key_pre+key)
    #print("dataset_I second page full response",dataset_I.json())
    dataset = dataset_I.json()['results']
    array_of_features = []
    for each in dataset:
        photos = []
        # centerpoint for coordinates
        #print(each)
        lat = each['geometry']['location']['lat']
        long = each['geometry']['location']['lng']
        # properties
        address = each['formatted_address']
        id = each['id']
        name = each['name']
        # photos is an array
        try:
            photos = each['photos']
        except:
            photos = []
            #print("photos , attribute error but kept going")
        place_id = each['place_id']
        try:
            rating = each['rating']
        except:
            rating = 'NA'
            #print("rating , attribute error but kept going")
        reference = each['reference']
        # types is an array
        types = each['types']
        testPoint = Point((long, lat))
        test2_geoJSOn = Feature(geometry=testPoint, properties={"job_title":job_title,"name": name,"address":address,"id":id,"photos":photos,"place_id":place_id,"rating":rating,"reference":reference,"types":types})
        array_of_features.extend([test2_geoJSOn])
    #new_FeatureCollection = FeatureCollection(array_of_features) 
    #with open('../DATA/'+nameOfSearch+'_2ndPage_test.geojson', 'w') as f:
    #    json.dump(new_FeatureCollection, f)
    #print("second page results = ",new_FeatureCollection)
    #return new_FeatureCollection
    return array_of_features

##### Below is the a function that runs through a list and calls the two functions above and combines the results into a single result

In [281]:
def create_List_Features(cp_array_formatted):
    array_of_all_features = []
    for each in cp_array_formatted:
        #try:
        features_1 = callGooglePlacesAPI(each[0]+"+Houston+Texas","bigTest2",each[1])
        #print("features for a company name =",features_1)
        if array_of_all_features == [] or array_of_all_features is None:
            array_of_all_features = features_1
        else:
            if features_1 is None or features_1 == []:
                pass
            else:
                array_of_all_features.extend(features_1)
        #except:
        #    print("error in except",error)
        
        #print("array_of_all_features = ",array_of_all_features)
    
    return array_of_all_features

##### function that takes a list of features and returns a feature collection

In [282]:
# creates feature collection from list of features
def createFeatureCollection(list_of_features,geographic_limits):
    slimmed_featList = []
    print('geographic_limits =',geographic_limits)
    for each in list_of_features:
        print("each = ",each)
        print("each['geometry']['coordinates']",each['geometry']['coordinates'])
        print("each['geometry']['coordinates'][0] =",each['geometry']['coordinates'][0])
        if each['geometry']['coordinates'][0] > geographic_limits["west_limit"] and each['geometry']['coordinates'][0] < geographic_limits["east_limit"] and each['geometry']['coordinates'][1] < geographic_limits["north_limit"] and each['geometry']['coordinates'][1] > geographic_limits["south_limit"]:
            slimmed_featList.append(each)
    
    
    print('finished the function createFeatureCollection and slimmed featureList is slimmed_featList',slimmed_featList)
    return FeatureCollection(slimmed_featList)



##### function that takes the return from the above function and saves it as a geoJSON file

 30°15'6.44"N   96° 5'17.13"W
  29°16'56.13"N  95°57'46.47"W
   29°21'28.30"N   94°38'24.37"W
    30°26'55.88"N   94°48'6.63"W
    
    north limit = 30°26'55.88      30.44885556
    south limit = 29°16'56.13"N     29.28225833
    west limit = 96° 5'17.13"W     -96.08805556
    east limit = 94°38'24.37"W     -94.64000000
   

In [283]:
# convert feature collection to geoJSON
def convertFeatureCollectionToGeoJSON(featCollection,fileNamePath):
    print('got to the function convertFeatureCollectionToGeoJSON')
    with open(fileNamePath, 'w') as f:
        json.dump(featCollection, f)



#### function that calls the others. It takes as arguments the list of companies and job_titles & the path to save the resulting geojson to

In [284]:
def make_geojson_from_CompanyJobList(company_list_formatted = ['puppies'],path="../DATA/test_geojson_gAndg_v1",geographic_limits=geographic_limits):
    #### function that calls the google api and returns results in terms of a list of feature
    list_of_features = create_List_Features(company_list_formatted)
    #### function that creates a feature collection from the list of features
    featCollection = createFeatureCollection(list_of_features,geographic_limits)
    #### function that takes the feature collection and file path and saves the feature collection as a geoJSON file
    convertFeatureCollectionToGeoJSON(featCollection,path)
    

### The geographic limits that trim down the results to only things around Houston as some jobs are advertised in Houston that aren't located in Houston

In [285]:
geographic_limits = {"north_limit":30.454961,"south_limit":28.956857,"west_limit":-96.206159,"east_limit":-94.64000000}

### block below should call that function

------------------------------------------------------------------------------
## Run Everything As Single Function
------------------------------------------------------------------------------

In [307]:
##def getJSON_from_jobsList_everything(array_of_job_Titles,FilePath,path=,geographic_limits = geographic_limits):
    #### returns list of companies advertising each job in an array of job titles
  ##  list_of_CompaniesAndJobs = runMultiple_skills_info(array_of_job_Titles)
    #### saves the list of lists from above... and formats it a bit further in prep for next step
 ##   Array_of_JobCompany_edA = saveArray_of_JobSearchResults(arrayOfJobsAndSearchTerm = list_of_CompaniesAndJobs, FilePath)
    print("in geoJSON_from_jobsList_everything, Array_of_JobCompany is =",Array_of_JobCompany)
    ##### opens and cleans the list of companies 
 ##   Array_of_JobCompany = load_listCompanyJobtitle(FilePath)
    #### runs the companies through the google places API
    #### combines the results into an array of features
    #### turns that into a feature collection 
    #### then saves as geojson
  ##  make_geojson_from_CompanyJobList(company_list_formatted = Array_of_JobCompany,path,geographic_limits = geographic_limits)



IndentationError: unexpected indent (<ipython-input-307-70e7b0b2da42>, line 6)

In [ ]:
def getJSON_from_jobsList_everything(array_of_job_Titles,FilePath,path,geographic_limits):
    #### returns list of companies advertising each job in an array of job titles
    list_of_CompaniesAndJobs = runMultiple_skills_info(array_of_job_Titles)
    #### saves the list of lists from above... and formats it a bit further in prep for next step
    Array_of_JobCompany_edA = saveArray_of_JobSearchResults(list_of_CompaniesAndJobs, FilePath)
    print("in geoJSON_from_jobsList_everything, Array_of_JobCompany is =",Array_of_JobCompany)
    ##### opens and cleans the list of companies 
    Array_of_JobCompany = load_listCompanyJobtitle(FilePath)
    #### runs the companies through the google places API
    #### combines the results into an array of features
    #### turns that into a feature collection 
    #### then saves as geojson
    make_geojson_from_CompanyJobList(company_list_formatted = Array_of_JobCompany,path,geographic_limits = geographic_limits)




Input_List_JobTitles = test_short_job_title_list

FilePath = '../DATA/test_jobSearchTermAndCompany_all_v3.csv'

path="../DATA/test_geojson_Everything_v3.geojson"

geographic_limits ={"north_limit":30.454961,"south_limit":28.956857,"west_limit":-96.206159,"east_limit":-94.64000000}

In [304]:
funcArg = {"ListJobTitles":test_short_job_title_list,"pCSV":'../DATA/test_jobTitleCompany_all_v3.csv',"pGeoJSON":"../DATA/test_geojson_Everything_v3.geojson","geo_lim":{"north_limit":30.454961,"south_limit":28.956857,"west_limit":-96.206159,"east_limit":-94.64000000}}

In [ ]:
answer3  = getJSON_from_jobsList_everything(funcArg["ListJobTitles"],funcArg["pCSV"],funcArg["pGeoJSON"],funcArg["geo_lim"])

In [306]:
answer2  = getJSON_from_jobsList_everything(array_of_job_Titles=funcArg["ListJobTitles"],funcArg["pCSV"],funcArg["pGeoJSON"],geographic_limits =funcArg["geo_lim"])

SyntaxError: positional argument follows keyword argument (<ipython-input-306-311feeca933c>, line 1)

In [289]:
test_short_job_title_list

['Chemist',
 'Conservation',
 'Herpetologist',
 'Laboratory',
 'meteorologist ',
 'Naturalist']

In [255]:
#featCollection
list_of_features

NameError: name 'list_of_features' is not defined

## previous error with full list with spaces in job titles:
_title': 'Research and Development Technician'}, {'company': 'Chapin-Hall-At-the-University-of-Chicago', 'job_title': 'Research and Development Technician'}]
each=  Research and Innovation
soup.find(id = 'searchCount') =  <div id="searchCount">Jobs 1 to 10 of 382,916</div>
soup.find(id = 'searchCount') =  <class 'bs4.element.Tag'>
---------------------------------------------------------------------------
gaierror                                  Traceback (most recent call last)
/Users/justingosses/anaconda/lib/python3.5/urllib/request.py in do_open(self, http_class, req, **http_conn_args)
   1253             try:
-> 1254                 h.request(req.get_method(), req.selector, req.data, headers)
   1255             except OSError as err: # timeout error

/Users/justingosses/anaconda/lib/python3.5/http/client.py in request(self, method, url, body, headers)
   1105         """Send a complete request to the server."""
-> 1106         self._send_request(method, url, body, headers)
   1107 

/Users/justingosses/anaconda/lib/python3.5/http/client.py in _send_request(self, method, url, body, headers)
   1150             body = _encode(body, 'body')
-> 1151         self.endheaders(body)
   1152 

/Users/justingosses/anaconda/lib/python3.5/http/client.py in endheaders(self, message_body)
   1101             raise CannotSendHeader()
-> 1102         self._send_output(message_body)
   1103 

/Users/justingosses/anaconda/lib/python3.5/http/client.py in _send_output(self, message_body)
    933 
--> 934         self.send(msg)
    935         if message_body is not None:

/Users/justingosses/anaconda/lib/python3.5/http/client.py in send(self, data)
    876             if self.auto_open:
--> 877                 self.connect()
    878             else:

/Users/justingosses/anaconda/lib/python3.5/http/client.py in connect(self)
   1251 
-> 1252             super().connect()
   1253 

/Users/justingosses/anaconda/lib/python3.5/http/client.py in connect(self)
    848         self.sock = self._create_connection(
--> 849             (self.host,self.port), self.timeout, self.source_address)
    850         self.sock.setsockopt(socket.IPPROTO_TCP, socket.TCP_NODELAY, 1)

/Users/justingosses/anaconda/lib/python3.5/socket.py in create_connection(address, timeout, source_address)
    692     err = None
--> 693     for res in getaddrinfo(host, port, 0, SOCK_STREAM):
    694         af, socktype, proto, canonname, sa = res

/Users/justingosses/anaconda/lib/python3.5/socket.py in getaddrinfo(host, port, family, type, proto, flags)
    731     addrlist = []
--> 732     for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
    733         af, socktype, proto, canonname, sa = res

gaierror: [Errno 8] nodename nor servname provided, or not known

During handling of the above exception, another exception occurred:

URLError                                  Traceback (most recent call last)
<ipython-input-215-31fbce623b6f> in <module>()
----> 1 getJSON_from_jobsList_everything(array_of_jobTitles=funcArg["ListJobTitles"],FilePath=funcArg["pCSV"],path=funcArg["pGeoJSON"],geographic_limits =funcArg["geo_lim"])

<ipython-input-213-69bd9f374f10> in getJSON_from_jobsList_everything(array_of_jobTitles, FilePath, path, geographic_limits)
      1 def getJSON_from_jobsList_everything(array_of_jobTitles,FilePath = "../DATA/placeholder,csv",path="../DATA/placeholder.geojson",geographic_limits = geographic_limits):
----> 2     list_of_CompaniesAndJobs = runMultiple_skills_info(array_of_jobTitles)
      3     saveArray_of_JobSearchResults(arrayOfJobsAndSearchTerm = list_of_CompaniesAndJobs, FilePath = "../DATA/placeholder.csv")
      4     make_geojson_from_CompanyJobList(company_list_formatted = company_array_formatted,path="../DATA/placeholder.geojson",geographic_limits = geographic_limits)
      5 

<ipython-input-43-99f3c0054790> in runMultiple_skills_info(array_of_jobTitles)
      4         print('each= ',each)
      5         #temp_results1 = []
----> 6         temp_results1 = skills_info("Houston","TX",each)
      7         if temp_results1 is None:
      8             continue

<ipython-input-42-9f17a4d1b584> in skills_info(city, state, job_title)
     89                 # Now that we can view the correct 10 job returns, start collecting the text samples from each
     90 
---> 91                 html_page = urlopen(current_page).read() # Get the page
     92 
     93                 page_obj = BeautifulSoup(html_page,"lxml") # Locate all of the job links

/Users/justingosses/anaconda/lib/python3.5/urllib/request.py in urlopen(url, data, timeout, cafile, capath, cadefault, context)
    161     else:
    162         opener = _opener
--> 163     return opener.open(url, data, timeout)
    164 
    165 def install_opener(opener):

/Users/justingosses/anaconda/lib/python3.5/urllib/request.py in open(self, fullurl, data, timeout)
    464             req = meth(req)
    465 
--> 466         response = self._open(req, data)
    467 
    468         # post-process response

/Users/justingosses/anaconda/lib/python3.5/urllib/request.py in _open(self, req, data)
    482         protocol = req.type
    483         result = self._call_chain(self.handle_open, protocol, protocol +
--> 484                                   '_open', req)
    485         if result:
    486             return result

/Users/justingosses/anaconda/lib/python3.5/urllib/request.py in _call_chain(self, chain, kind, meth_name, *args)
    442         for handler in handlers:
    443             func = getattr(handler, meth_name)
--> 444             result = func(*args)
    445             if result is not None:
    446                 return result

/Users/justingosses/anaconda/lib/python3.5/urllib/request.py in https_open(self, req)
   1295         def https_open(self, req):
   1296             return self.do_open(http.client.HTTPSConnection, req,
-> 1297                 context=self._context, check_hostname=self._check_hostname)
   1298 
   1299         https_request = AbstractHTTPHandler.do_request_

/Users/justingosses/anaconda/lib/python3.5/urllib/request.py in do_open(self, http_class, req, **http_conn_args)
   1254                 h.request(req.get_method(), req.selector, req.data, headers)
   1255             except OSError as err: # timeout error
-> 1256                 raise URLError(err)
   1257             r = h.getresponse()
   1258         except:

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [272]:
copied_feature_list = [{"geometry": {"coordinates": [-95.46103649999999, 29.737203], "type": "Point"}, "properties": {"address": "2800 Post Oak Blvd #3900, Houston, TX 77056, United States", "id": "4b62e79323dd04edd16f18a3570c6c525c72352a", "job_title": "geologist", "name": "RBC Capital Markets", "photos": [], "place_id": "ChIJMzyxy23BQIYRmQh7_lj7cok", "rating": "NA", "reference": "CmRSAAAAZHCxUrcoNdmNHk-acO8JGRqvdpVGfoaJwc8gocBrMOwPY_kNrdG_voVRQhyobLytDJDE3z4no1B-jG-S1UU-XT65b8VH-a4MgXx5-b2DqbRErnd959mCH-Gbr5Bu7ptNEhA-to4Cld2aG366VDyB5m5RGhSDRAL2FObJKSL8ZwimNGfoWI8D1w", "types": ["bank", "finance", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3640089, 29.7604054], "type": "Point"}, "properties": {"address": "600 Travis St # 6500, Houston, TX 77002, United States", "id": "e6f1fffeaff13624b25a14bca2c17e581e517245", "job_title": "geologist", "name": "Royal Bank of Scotland", "photos": [], "place_id": "ChIJa0oElDq_QIYRxrrhsSUehHU", "rating": "NA", "reference": "CmRRAAAAksP4-7nK7sXWDuwBtg-EXiHWpJP_yUy4amXqPNYGyb47XGiTkyoq1rtlPzPnB2OEQUbHIttoRPqQsIM1Y9ff-P9JUIXDmLQbLAj-bLSYGVYbhbQ0lPwAPj9qlE3rrcg1EhB_jTltDy_44fdiWUeUh47LGhQ1pFYpvYFB75tv6AT6QDSNCrk5hw", "types": ["bank", "atm", "finance", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.56025, 29.781615], "type": "Point"}, "properties": {"address": "#300, 818 Town and Country Blvd, Houston, TX 77024, United States", "id": "a674ded7a91a457c3d42b551c5d48886ff6b71a0", "job_title": "geologist", "name": "RBC Wealth Management", "photos": [], "place_id": "ChIJK5dbXLPEQIYRQeq5MViNqig", "rating": "NA", "reference": "CmRRAAAAZcdsVAzSrtHD-Tj0udLhSS0SDH-UPzUPYMeZ3Xha1UzvK8Ppxc1ccMI7rjclDSnWcir1y-L1n0HtkLZgrzndU6PgpPcOwI79VM-azj9OvyJtosgAOuO5JbqgKzGh5ibUEhCR0900CNIPoFVohpI_AcLDGhRyMIElqrWc1Zkyhn8VxH7zj5o-lw", "types": ["finance", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.45170759999999, 29.7477721], "type": "Point"}, "properties": {"address": "4400 Post Oak Pkwy #2400, Houston, TX 77027, United States", "id": "ebe2836d9d530f6081066f7a4aed3a7d4066d74f", "job_title": "geologist", "name": "RBC Richardson Barr", "photos": [], "place_id": "ChIJX1ZfCT7BQIYRrP8hSoM5Mpk", "rating": "NA", "reference": "CmRSAAAAoFk_baD9zBngvpBzuhqLGLD1R3VjijKb77urDytsLaGcHqyLg0gKRJuowObb6hvn6CbpADCmSCuK_Ntmvxigto9LUgzNCWkaIxZPCaGWrz_q32TDWMzcvDjJBWt3BvedEhDK1etcP722mC7kplFZeSmHGhQN27PHTjczUbck20wmg8_7eKGq1Q", "types": ["finance", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.461597, 29.746341], "type": "Point"}, "properties": {"address": "1980 Post Oak Blvd #1630, Houston, TX 77056, United States", "id": "4eedac6f1ba8c81d62786ea3ea10f3686ad5b9d2", "job_title": "geologist", "name": "RBC Wealth Management", "photos": [], "place_id": "ChIJlVxoBkLBQIYRDDHQy3MpT_A", "rating": "NA", "reference": "CmRSAAAA1YorTOPuGmoNXLBvZe027qM6M-kf28IIAEXp69BmpqAqhganReouO6Nc6SutM4unYNBodk8EZIFkLwUPSdqv4UuvjQtSVD7yoip4DMOqzuR_ZSQPMjZP1jQC6md8kbpGEhDXGKhJiJA6OuM_VOHnSFD5GhQ9AEnBaVP7XnJnfbIQ67JhMcXBxQ", "types": ["finance", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.36388699999999, 29.755906], "type": "Point"}, "properties": {"address": "1001 Fannin St, Houston, TX 77002, United States", "id": "cd0e0c0273242313933c8043aec4389b3fb9a983", "job_title": "geologist", "name": "RBC Wealth Management", "photos": [], "place_id": "ChIJr9ZsYDu_QIYR8bROl7dKBws", "rating": "NA", "reference": "CmRRAAAACVEOnQqw7dIbAC8kannYNuUPVC2k0V6VOepTzJP4w7xik0_azttSiwPyaFryHIplmnVKaI3t4alQ8N5pjkjaLk4SgMNvwzB5dlXJWih5OvstPoyfRTGK8wrjhj9PZFgtEhAhEf0sRguaQykZvkocerF2GhR4CBJzH9cc9d3H7TtEKRmOU-Y3fA", "types": ["finance", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3599019, 29.7546294], "type": "Point"}, "properties": {"address": "Hess Tower, 1501 McKinney St, Houston, TX 77010, United States", "id": "04c70c4a3d18050ca437bbaedeca70bd9cb01927", "job_title": "geologist", "name": "Hess Corporation Exploration & Production", "photos": [{"height": 1454, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/111482964284093817946/photos\">Sau Hei Lam</a>"], "photo_reference": "CoQBdwAAADAsGx-uYzaxWlUYRcDGRkqQsXvJslT-KiIKXYL_ZfuvPnpctDeOJAP2Twjmwh9EWZv98IW9t5ZAKPXNzYIL5yQMqNMhuZfW9ImQd-_-nF2-g_y4ItQ1v7V960m_6DgsLu0cEMlCRIFnzJFBvjedr_NAPQur3J4ZvvXjFue-ktxLEhBDDUD6PSd_Kht31zarvLvFGhQuGlp8eVWrF5Y10-A_x5ge6omCbw", "width": 2161}], "place_id": "ChIJfy2iBDm_QIYRjgKEE3GFL9Y", "rating": 4.4, "reference": "CmRSAAAAyjqRBy1c1Gl9pVkQO55DOWnlufEOpBacjpDiMWE9Nhq5it-vGlmjyOiaWfl22foiZTngB4QB04OFlLYF9AkpUuqlPxtxWhq6GDkruNKGA3IBFS58fQReTZkxZW64SOLxEhC78Fkoh_OKppDszKIhXz7OGhTiCjCr3EtrFQET5AbHpoGot4_hDQ", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.53667899999999, 29.643324], "type": "Point"}, "properties": {"address": "10710 S Sam Houston Pkwy W #3, Houston, TX 77031, United States", "id": "21ef45fcadb15a9dae92a34c0882930987cfd934", "job_title": "geologist", "name": "Tolunay-Wong Engineers, Inc", "photos": [{"height": 785, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/100502235174758387817/photos\">Tolunay-Wong Engineers, Inc</a>"], "photo_reference": "CoQBdwAAAArT1CGFRoNZlFj7WQjUXK8QliuPYYi0ukbmGisMoj4h9t5BDd20oeOfjQ-6BhBtIR66MyTx7cLCQR3Dsc1OtnxU6uqD9Zd7lILGYyrzHSX91Fa7Qz8EOGg-_CH4xVGf6dUnbEdqDy7WHZoKwadqEu3x_Bd3J1mLbkFLM6MOtdxAEhBa9c8gJcMW8-hVZraN4NrpGhTeX4STpG2LelsRPEeP2Nd_mJqT_w", "width": 1791}], "place_id": "ChIJgd3o6kPoQIYRBdEf5LXmjLY", "rating": 5, "reference": "CmRSAAAAM8g47276syZWm3MUBgnjg6htvM_G4JNfTnO7UQG5cGZNUG4ph-dPjjtEy9CKA379yU67_AX3ZkEgMQ5KXFg4U5JywdB0mAzt5lsbDk-eCw_g1JHyFhHucRLf2ATtXJA0EhBTc6rjFTKJol2OR7vhJ3iqGhSKPbBcxsnIJn8qV1ieX2N2TbMnDg", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4620181, 29.7530878], "type": "Point"}, "properties": {"address": "1360 Post Oak Blvd #150, Houston, TX 77056, United States", "id": "9aba7fe2f8a80c94b359e1fd5c4df4b3c3e70314", "job_title": "geologist", "name": "BHP Billiton Petroleum", "photos": [{"height": 1724, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/106919442231730187134/photos\">Chad Burrows</a>"], "photo_reference": "CoQBdwAAAG3bCPXw7iKaUG4o8aFuoAtfOTpTBN7ohWBOjyOfuu6IQA7guxb5xbBHoMfDVna9zRYFS-wJdyNM-RbFCP7eYU-5L0G61HdFBPDWEygzPeN2LVRJqfzXdBiPAQsP-Dem0Demm86xDUG7zdPDr1ZwQdUjoN68LiskJ6M0o1C_32dSEhDb80JgBRumf-2Dc1vGpUVoGhRxN-dDL89DmubVUc0AEVNkqd7moQ", "width": 5502}], "place_id": "ChIJU5hfp0bBQIYRKqHNd5nxNbw", "rating": 5, "reference": "CmRSAAAAOtuDI97DSC03CfMFYMeMM7Le4JVH32Ue_zzO5kvfAyiE4iB8sv2lsI_-nGP6NTYxC0-cLgLk1xKCcsTIRKK54AnR0--cAx9tIIGzY-e3AP4TgDIU0A2-1lrajGV4L9fxEhCdYg1w1KdB43tyM5fl1LonGhR9xxRxy2aLuY_vyoK0hOS5VjHMMQ", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.1974345, 29.658069], "type": "Point"}, "properties": {"address": "1006 Vista Rd, Pasadena, TX 77504, United States", "id": "69a96533a258eb591726d86d1ca2f9c6ec0228de", "job_title": "geologist", "name": "En Rud Resources Inc", "photos": [], "place_id": "ChIJc_A7nUSYQIYRcN8s1islssY", "rating": "NA", "reference": "CmRSAAAARNn7XDmBjchus_55SmPZ5TmGnW7ArXqyAQxN4TgOLGP25tcDU8f2gTvRAIiN9u1d5gbYaBRUoTB7srB9y-2X7juZxovmAIefNQt0qRCCcS8xORxkF-I1j4rF8I9IY-w7EhAqQwfsoQ7OwFTBZHTEwPJcGhTtR6GTosBmPdYpXthBzXDr6knVuA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.1721678, 29.64837929999999], "type": "Point"}, "properties": {"address": "4624 Fairmont Pkwy, Pasadena, TX 77504, United States", "id": "9ae520cccc7564f96afd46f6d631d61bdd6bdfff", "job_title": "geologist", "name": "San Jacinto College - District Office", "photos": [], "place_id": "ChIJ5czzhuSYQIYR5xO1zKMXD90", "rating": 2.3, "reference": "CmRSAAAA4L4AmA9IycF2CT257U_Hh7eozu7eVLnC9T3GlQw4NtraevdReLy38PMYFI5IwwYPfrq-MahL1SrJynGVETVgaaSfogNOhbCiIYaIXhpiCt99j3M_f6j7e1ag1muR_O9fEhB2bxYUhExHFmwkvWaNpYTTGhRm2YBVNSsg1RuZnnPOlyyhsQ1_Cg", "types": ["school", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.366643, 29.76074], "type": "Point"}, "properties": {"address": "700 Louisiana St # 2100, Houston, TX 77002, United States", "id": "ec8b82b104863bab3b36a1ce79a0c0f711468bfc", "job_title": "geologist", "name": "BMO Capital Markets", "photos": [{"height": 750, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/104549896185334272100/photos\">Duplan Nalpudi Instagram</a>"], "photo_reference": "CoQBdwAAAAJ6XTFHXEE0Wadaxe8OTluFE7HUrb0Hq_EBeBBivWEwG6vKOzetGq_2z7PWILXcRQy_EIbhdSPZK2vbrxRHH9IFCS14fkscmI6RNIPFXi_vSGjA1-YMMP1QmIeNlfP2W3_hTd5Ay8If866LrdeVmHM1_tLMRs1u9rYHR0tQNTDyEhCzS3KITjIfGZJIXYxmW41QGhSeJURd4g9XdNAeTX8C48oWmAXjOQ", "width": 1334}], "place_id": "ChIJLROgDTq_QIYRTwQmXrl5Wm0", "rating": "NA", "reference": "CmRRAAAAxnASvDFOUjGrpkDYDWmqNrv5OaQE8PPMqfWuJsESxbIPLkcapX38R6GqZmBN9EqoPAJ8eFC15N7smXVcNe3avTJYlmaRh7i34FLRAD-6ak7H_zs5g401_5P1MFqd-MCtEhBrI68CWVZNFxuNb2weMXfTGhRj8jGjRvBUfk5gIjIeU-NqkPS5mw", "types": ["finance", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.372164, 29.757501], "type": "Point"}, "properties": {"address": "333 Clay St, Houston, TX 77002, United States", "id": "795a3515ff023985c96d55d60c8eefcdc6c9ee0e", "job_title": "geologist", "name": "Jefferies", "photos": [], "place_id": "ChIJe81fkji_QIYRhzwdUhh-ygw", "rating": "NA", "reference": "CmRRAAAAyEvFORruDkPxY6ltJiQcOqA9ih16L_Bffm7-I-zSf8p24oX3ggzEZxXflRPr0MkIbr-8CZPbrKTBX9ldF7ApeJhHCf-yAXZr0Q-wKPqacQFS2LftiUsbgGRwZ10fq8CBEhAI8nEAl2mIFQhwWyUpwknyGhR6dWz7jSHNRyWRwkpalAvStqE-SA", "types": ["finance", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.49726899999999, 29.7140059], "type": "Point"}, "properties": {"address": "6100 Hillcroft St, Houston, TX 77081, United States", "id": "372e0791c5d5ea0bf4c3a24c278c5083aad17624", "job_title": "geologist", "name": "Fugro Consultants Inc: Akhtar Anwar", "photos": [], "place_id": "ChIJ5e-dPzbCQIYR8o0_0WHFuVk", "rating": "NA", "reference": "CmRRAAAAJfSbIwfGz4EogbvyegiWN7kAXAqG-eCvMKewYZtyW9qGnal_gIO_Muj_4gwxWz3-_IQ-R4zMI2pDFS3Hj6dLv4vqQGbnTjQd4HUVCHIuXuxqFjKF7xJpNy722nji4ElVEhDXLF1LrlDT4a3IlkXq-JDjGhQPorvUROrFmF4T1ejh9270ZKgk7A", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5678336, 29.8452744], "type": "Point"}, "properties": {"address": "10831 Train Ct, Houston, TX 77041, United States", "id": "94c4bf4197969258554d1555fd7ce686908546b7", "job_title": "geologist", "name": "Fugro Consultants, Inc.", "photos": [], "place_id": "ChIJ1WVb4rHaQIYR6O9DfGla99A", "rating": "NA", "reference": "CmRSAAAAKZUmGmU-bf7JcLeqY4CoAO4bJOU9BWP1WCw87_vXy-_3agr7LZX3mch5WO-iWqlEqkZsAePSh7Lk5AN7BILjLmAew1GbqUoG_JPT-k5aDxcFNWdwpJciOb2BwN5VV7dHEhBoD9R3HF4PMvor-5dmDx0zGhR0WBaFdbCL5QEDKypbe9V5mVMSog", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5441765, 29.9263062], "type": "Point"}, "properties": {"address": "8715 Fallbrook Dr, Houston, TX 77064, United States", "id": "4b8b00e13f61a661128d24dcd3068ddd462bddbc", "job_title": "geologist", "name": "Fugro", "photos": [], "place_id": "ChIJC49COz7OQIYRAkBflL00eJ4", "rating": "NA", "reference": "CmRSAAAAKnJ-PLXUz2cGuMXBqlupgPuRmIx6br2jerJOHQ3PnCMCu55xBuGY8L0U1eAO8sSteu3nmHbDVGVgstxk-rmM-VLSZlwkm1n3H43KdgX74rVcp7X52uM0WGv-49M_qALbEhBU-hqOBMDdyEaTaiUxIIE_GhQOwIsvKrQrYjHq9q5b94aoR87I2Q", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4994463, 29.7127126], "type": "Point"}, "properties": {"address": "6121 Tarnef Dr, Houston, TX 77074, United States", "id": "036ef56e28e7616b45367256404bd34b6894d17b", "job_title": "geologist", "name": "Fugro Inc", "photos": [], "place_id": "ChIJ1ZxaK0jCQIYRkHmkjNdFW2I", "rating": "NA", "reference": "CmRRAAAASHUZ9MB81iK2MLEhgOYZ5Eru9ekeNs_Vl8F91I6mVP3MX5_ce48QOzNfpxbB74_ctpGO1q_B3m8E8herIETMU64Y9ACmVO_HjXTq7cEHCWkl8TpOkJrLiYA7R5cRglE3EhAVDmbN2jituXKBsYShQRzXGhSJHn5bD7k3BQxxX2w2x08kBUtNgQ", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.49562680000001, 29.7167202], "type": "Point"}, "properties": {"address": "6080 Hooton, Houston, TX 77081, United States", "id": "65dd391d8d93517cc6dbcdc22dc4231b9aa5303c", "job_title": "geologist", "name": "Fugro Inc", "photos": [], "place_id": "ChIJrdJGMjTCQIYR03_Oibwj_uM", "rating": "NA", "reference": "CmRSAAAAowhSnsCeJm3gYOnEDNZ7a43E-gqlh62gNyk2xMRNqkO9GoQszFgAI5RFPL8zCrlgZAdbgyqoPFlmMVqxvGGkLT6by6pTBuN0BeT3C-L1AD41SfUVOy4NiTG7LHNngnguEhAgJX7Xs8ZVeX7kSXgb14npGhSlkRKhK8KPDxnrwSC9OPxAoI-36A", "types": ["premise", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5503257, 29.8584718], "type": "Point"}, "properties": {"address": "10425 Okanella St # 100, Houston, TX 77041, United States", "id": "bd7b9e455e61e36012f43900a5d2dc8fb520b286", "job_title": "geologist", "name": "Fugro Data Solutions", "photos": [], "place_id": "ChIJf7sDHvjPQIYRNi_3faTzFrQ", "rating": "NA", "reference": "CmRSAAAABMsN0BF4qImRGXhbd9C3pv5Wpuh-qu1oMpNEsG3IrcMLS9g_2trO-QYxZO-PhzmRqzVXFSRO26iD97rvlaU-OicQK_eO2rIkbAGxqqQXLhrIelEF4-iFbH98WgNl5z8bEhCueCT4nussffuXOMwuSymqGhS7CJHFuQ8i4pzMsHSL24DrOdjSZA", "types": ["storage", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.1800995, 29.7118821], "type": "Point"}, "properties": {"address": "2410 Pasadena Fwy, Pasadena, TX 77506, United States", "id": "38cb6e415101a4bac2c1eaf17a6fa77679f6788b", "job_title": "geologist", "name": "Fugro Consultants LP", "photos": [], "place_id": "ChIJQdSxQV6iQIYR9uoIbHWfBXE", "rating": "NA", "reference": "CmRRAAAAgRQU8gFQoDoGvfZONgFJ0L3TE7-2iinmImlY2HSudOYS_Z9PG8bUuZeql0kJn-HnUfS7S12okDg4LlsBofHxRZp8I1wUNbp8PkV9K2nC4wMRC0NaG7ow8FLyopwm0hRtEhDIEHytQwc3dPTnnxNSqkNuGhQK-5zmh5oXjG0Xyyo2LYd_VqRMEA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5740797, 29.7373078], "type": "Point"}, "properties": {"address": "11200 Westheimer Rd #365, Houston, TX 77042, United States", "id": "328101c9c23ad52a56a549ad72e19915b58d7af8", "job_title": "geologist", "name": "Walker Elliott", "photos": [], "place_id": "ChIJz6iXr8HcQIYRnzHDA7T8m-k", "rating": 3, "reference": "CmRSAAAAq3X0LE0-YJu5gH53VpxVHV1FRyfgqS5Tbzy9VOevK-OPO0E0edNWw5saqHYmTQeI13OrCRRn55IXCNR-twRG4LP9H4OLloCM1sXuLd5QKQfMBm1vsQJ62ahiFOynW3k9EhAnqQFCzrgoaAj2O0GjiknZGhT-CTEzUydto4xQHLi6leHqjz8IrQ", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3662701, 29.7561894], "type": "Point"}, "properties": {"address": "1111 Travis St, Houston, TX 77002, United States", "id": "6cf1f2b19f724b7014c7cab1c01725834dbc3db4", "job_title": "geologist", "name": "Hilcorp Energy Co", "photos": [{"height": 2448, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/100318015046809414765/photos\">Ronald Payne</a>"], "photo_reference": "CoQBdwAAACD4TuMx3YdUMuEMttal89_H2i4a_Yb_N8J84PvSwa6P7yZIPVzHSgLp0rVq9wVisW1zylOPPewgCqZl6wr9wB0CM8oikmhhBNk63khPmUvev2mLkGW1mnm1FE8hL4zqpLAq8mpxmVjo4VXFsg843Vhn6wmTnykgHrOvGdiL21RwEhCbTDT6gPAjAyW_adOD4KfvGhRWkL9T-I38Se4XCduhw9giN7lyaw", "width": 3264}], "place_id": "ChIJncGsRDm_QIYR0e57PbvugTc", "rating": 4, "reference": "CmRRAAAAR5xWpcIzDmwYtGOFcalgmHc9XTor2q_3GqgNBVbk-rIr9uOmpMFBfQ0rL3SO6sPVxjD4tLB0DzBSy5SdxAUJhn3WNEwdyz3Db526ujmfe-cO5iOFp8TQBWRJbXf4yY_aEhCjB73M7KtpHi8zSO7Hd1LHGhSIn_gKCLWU4j1Fho5aOjPkuYkMUA", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4321022, 29.7305064], "type": "Point"}, "properties": {"address": "5 Greenway Plaza Suite 110, Houston, TX 77046, United States", "id": "7394562c1f1a87dc4b6fa017b44a961df160f5ef", "job_title": "geologist", "name": "Occidental Petroleum Corp", "photos": [], "place_id": "ChIJjd5k24G8woAREjjHjBsq0Yk", "rating": 4, "reference": "CmRSAAAASz86jQdTDOwlIvwdnA3Qz8xUhez7ibpthJLs6BCJvA22JiaWnbdsJFy60jilcgoM0tpbN3i8KywPW5oMHvzxeGJM3LU3vSeg5MJRgwCKCL_OCqpGZllOudVGMM73GshlEhDVbtotSzfF8JxecMlatoslGhRp2Fwf93AvjW-Hj7Homm7sQj7oEQ", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3970411, 29.9417677], "type": "Point"}, "properties": {"address": "363 North Sam Houston Pkwy E, Houston, TX 77060, United States", "id": "bcfc0554855f942f730466fd447878511b6aa0b4", "job_title": "geologist", "name": "Occidental Petroleum Corporation", "photos": [], "place_id": "ChIJx73rqePJQIYRPjss3jFElZo", "rating": "NA", "reference": "CmRSAAAAhuoL6RZDwAgakjHAPkQ70yt-z6OkbCizb8q-DZhXu4VSVrqxw-9P1Z522eoJLNvg28otlNy0gfxz8RSdtk6_MVVmvbH_PrQiIAWCIzsxJGgLYBqO6wh-rYaQ8D6Qnr8kEhC2y74j1TpWgjqBhkoWFQXyGhSffVxx7DkI0kLRszfjmBpxHWXhBg", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5498597, 29.66888299999999], "type": "Point"}, "properties": {"address": "4301 Southwest Fwy, Houston, TX 77027, United States", "id": "6e40f1d16226178cc0cf90f68d007ada7cfe5617", "job_title": "geologist", "name": "Oxy USA Inc", "photos": [], "place_id": "ChIJj7uu3g7BQIYR1FyPVqiuar8", "rating": 1, "reference": "CmRSAAAAmEUCNeVc4zYeUcO3FYggAj1XDwroO90SL-bxT75fmTME5_iVd377kIQmhSdMrHrEWxyiU3ooZGVVjAnEqAnKU055hS5Jw9FjceJqbZqevK39LAgxXXVco9W1Y9Fl70-lEhC6j0QARdoZ1vWdm-jQ3M6sGhT5xXZn0-bzlUcIc7yrGTUwkfRXag", "types": ["gas_station", "car_repair", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5438387, 29.7776847], "type": "Point"}, "properties": {"address": "800 Gessner Rd #170, Houston, TX 77024, United States", "id": "91282085c6930276af6f7e2abf1ee0e0fc18432c", "job_title": "geologist", "name": "Murray Resources", "photos": [{"height": 683, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/106500950429449697681/photos\">Murray Resources</a>"], "photo_reference": "CoQBdwAAAFemj3kmrLnXYfPLodQxo5ElybQv0GkKavHa_44R-FibcTwjLE3xnJqBOVAXU_mwTEjQ3lQAzZ3nAhI_G-3-miEKPAqCWv6hujQV-DbyoXyUQ5wo-7_-rC0IKA4pp9l0k3cIKzZo-zTxE2B9D-a6zZMMN_BMnKH5x17GXA3bkgFvEhBZI3TaZ9CQN7454CvQeDuaGhQsBW--A7BjExS_YRHi0pXA_NslvQ", "width": 1024}], "place_id": "ChIJRSYpxLzEQIYRsKFA9Ahm-Z0", "rating": 4.6, "reference": "CmRSAAAAt8uZVNFLsZhKkIklOc3Wt3UL4i9ODr6P4gfKwLgUwKriPq-3BEX1keue__3Ln_57cVzDGSnrGBy0C4i2yYDBpKzRg2nDmA-MDpidWSGseEnsgC2T4Tj3E7i7bTG6Vb48EhAGh5UF-BJ_JIoZsGL08p8iGhQD3UQOpAbsKoGsdF7XlJjkykBh-w", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.411982, 29.838328], "type": "Point"}, "properties": {"address": "707 Lehman St, Houston, TX 77018, United States", "id": "e3a902a8d57c6ee9d36a8e1a32fa39c463a33a0c", "job_title": "geologist", "name": "Intera Inc", "photos": [], "place_id": "ChIJI-1445THQIYRvn-6mbb_i8I", "rating": "NA", "reference": "CmRSAAAAjMFpeODpK4DnExOwKuas0jsuXqbP1uyqyvt2I7lDsxE8H6wdt4GeLnF3WpZTmhNzX8XaMh9zktB4PRRW3khfr9aKWyHQkdDdsZYLLj6_TydX0GS0j5POQ83q1SkA-cj4EhAgpn3QZGXi92tnkS7QDK6uGhQmpNusZ0uoD9L5vrwTJiYidnHgSg", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3574139, 29.8867077], "type": "Point"}, "properties": {"address": "11939 Aldine Westfield Rd, Houston, TX 77093, United States", "id": "596dee2eed3253cb2fc6ebf53ac71041aa3694f7", "job_title": "geologist", "name": "Weatherford", "photos": [{"height": 2988, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/107786689521565648284/photos\">Donnie Scott</a>"], "photo_reference": "CoQBdwAAAPBqgpJau23JGWsXP_aHT2dZ0z6KNpoGUZcFHUQooyiDqK2DTxBjYbTlPN3zY6r6RUJ6DSpqhAP509_QDUFJKIDgU16rpOmH2-dKijS_TyTlMKoLAaQSyvgJZUqZN1MdLdDtOM_DeR2dUo1Z6zlCSLpWvyMO0RvtcvwgKR-MG3wiEhAzflC8VBBqexJpo763_wKyGhRYW4LyCA1VYpi_0dgsU8iun9uAqg", "width": 5312}], "place_id": "ChIJJTOHIp-3QIYRLKxLIjOV8yk", "rating": 3.5, "reference": "CmRRAAAAszrI9qFjAvDkRZHAlfYcvBlgyGFY02Yn1xwQC3DyDlUrZUlLrCnRa0gxaiIF_gZ5UOCmm5X4UwcqL1VqZa73C70WPifaxR2ep5pRJ5VOeC_njU_SlJubAWzh1m2mIiTdEhBrqSrS-vOnX9_K3M9vT2iQGhRL2k_B3y8OnhNz4N_AwvVFEmLEvA", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.47312559999999, 29.74571959999999], "type": "Point"}, "properties": {"address": "2000 Saint James Place, Houston, TX 77056, United States", "id": "52afd1ae1d4a85581d56feb80c02e3a0f203e7c6", "job_title": "geologist", "name": "Weatherford International Ltd", "photos": [{"height": 5312, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/109977538253417550023/photos\">Toufik Djeradi</a>"], "photo_reference": "CoQBdwAAALEJd3HLtOFVf742uAUKfcWbY12zZLviR42hY1WctUwxrabZhWal2JR2pBP31fD44TTTvazBySu69YtJeMhLMxcz_E99YbxXe781BCYCab5ZzyeVWblcKVUoJ-zuzWdNLka2puviqeGFXQF02yzXQZ0LSij5nnmTB9wDRDl3s7TTEhBsFAtQlnvtj0D0xwshAcO3GhTFfXlAf0fVNCFrfEAUN2JlrzZENQ", "width": 2988}], "place_id": "ChIJ4ZM-A17BQIYR6GEbiv0fqjY", "rating": 3.8, "reference": "CmRRAAAA0S8bhOc5AkWCHfkYXa-8GTUlGY4JanIPmTlYfPWAGl3iEX9eaYNLKe6RSO8lNSFmnn9psfeWPY8-R8gfjAr0u6dFzgox9nh4x4OG9wg7OyfFnQFJba0VcHbFazmlGQ3LEhAkWcdb5kxLYW4qbs0Mym1lGhRgVsmkE-hBMRISasdiXPMjJPsREA", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4008035, 29.9623403], "type": "Point"}, "properties": {"address": "18160 Imperial Valley Dr, Houston, TX 77060, United States", "id": "4d3095b9bb6426aacd6c3a9290d25e7a1271e552", "job_title": "geologist", "name": "Weatherford", "photos": [{"height": 2268, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/103289608261407307137/photos\">KEVIN THOMAS</a>"], "photo_reference": "CoQBdwAAAFkUHqa-ujnW7m_GyRGOsFG5J2zTbQLlbhT0hFJD1W6oVuYrAVbBvQUjxPfyZYaPdBfsMznpVBkwljhPWsEQffrfUt1FZP781ZcS2dSz4z1sjhWfoy13wz2RuhXftdP04xiu8I5p1eEiucMmArsUtIbt1xI-OAOYi71SkQqi1f74EhB6bXxL2tCY1EOfBbm7r0ufGhQHId0-JKkc804PeKHtTtkUYl7y0Q", "width": 4032}], "place_id": "ChIJ0-giCwTKQIYRcRakt7fNf8Q", "rating": 3.5, "reference": "CmRSAAAAg1YoqU8G0Mvx4FMNxoA3twXlk20iaACySpBEAPk6wYCCXaKhSB8MaBhj7N5XeqWHELCUBvolDsarzM6v50xMGU-1Hax_5ue18KgRm29GABrj0w15DN-UUHRGK9hqj8BKEhC9n-bnAqSHI8AXVvcz3AbnGhQZbJrYst_4BFnMVfb1O_Xodzy_kw", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.59471649999999, 29.8775974], "type": "Point"}, "properties": {"address": "11997 Farm to Market Rd 529, Houston, TX 77041, United States", "id": "952ed47a8b304b500d4c7fa4b2f7acc9d81862d0", "job_title": "geologist", "name": "Weatherford", "photos": [], "place_id": "ChIJN6mSK2jQQIYRsomU7nEAtQo", "rating": 5, "reference": "CmRRAAAA-t6v6dX4zv_WrZqHf2TqiE3366W1QBzJE_Rw4vDukPPEi2Xd9FUT1d8As_530LtVxZuL-tgEIXrdReMXuK-NRlpP1fcc6HlwP5jLltUnTNOFx2tYsLjDC9jfrlTFgq7-EhBs8VeTJ4uNzFS0BQm9RBHsGhQ7ainQBAbT4n2AiK118M2zio1ihA", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.50646259999999, 29.8392312], "type": "Point"}, "properties": {"address": "4409 Clark Rd, Houston, TX 77040, United States", "id": "5897514914025020b16834cc3321b79cd91ddcf4", "job_title": "geologist", "name": "Weatherford", "photos": [], "place_id": "ChIJRavrVZbFQIYR-V3u4J6Px6g", "rating": "NA", "reference": "CmRSAAAAs0UsPlWE2YHivzNIhYeQOD3TfOHJVJhxSPpDZvIOb-FrRJ5zHwfDqMcT9nUkVZt0LhZikWEl83dLKmJMYQvOjENeJChA7IV1gkeoE5EgfWPO1KO5JW0Zj-3EfLz5X_5-EhBRBfg-qJMiazKTtLIZkuX9GhQvER37KjGNIJW6GmiByot2Kr6M3A", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.38671629999999, 29.9536867], "type": "Point"}, "properties": {"address": "16210 W Hardy Rd, Houston, TX 77060, United States", "id": "d8740eea0d7d20abe22e8ee366dc5c88884d8890", "job_title": "geologist", "name": "Weatherford", "photos": [], "place_id": "ChIJaSSBtgC2QIYR1MT_9TLRMdc", "rating": 4, "reference": "CmRSAAAAJeocjUCbVFNTrdH9o7drPGzijbHbjt6V8BmAJ9LHtw3NsL2CmIZx3y7LJOspnUzh-X5V2GTpAN8NsMbnYylkI1a74HAhCdn-__zo7drZTfn2G1e4pidmrztwGVUKlcrfEhBeSCmfWV3qsQ_HK5j7AbnvGhS3toK73knwKZ0kijFdJEahSjj0Vg", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5414947, 29.8793251], "type": "Point"}, "properties": {"address": "9600 W Gulf Bank Rd, Houston, TX 77040, United States", "id": "1410b97b0d3e49fac99b37306c2f239edf2cbfcc", "job_title": "geologist", "name": "Weatherford", "photos": [], "place_id": "ChIJIYP0x-rPQIYRQkqUN2zvUVk", "rating": "NA", "reference": "CmRRAAAAv7RdhIoNfr9JmbOql7CSSVTJeJGLhC6PL6zgxR5bzqAeyhFugLsns2qIap-_VGjJJZ2-ySyFqzAFZAeZLqlE5jgz0SUbymucAPg9GUe6teId0ZIrjTOhRdLLps8wUb7AEhCm_Ggjv7O12huk9exoPvdAGhT3I0042XMYziX6EnPpMO7Ptz6gMA", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.56310339999999, 29.86431019999999], "type": "Point"}, "properties": {"address": "6550 West Sam Houston Pkwy N, Houston, TX 77041, United States", "id": "6af716c199aa85ab00b3099d81170d6589a15220", "job_title": "geologist", "name": "Weatherford", "photos": [], "place_id": "ChIJA--CRgLQQIYRlczuvd0DaDk", "rating": "NA", "reference": "CmRRAAAAnkhj-G6aip3CKIIRHwoeGEwezIyeCoiFJvOz-ndNVYL0ABj0g16nYU5MQb1qB8JqZaj1NCLu5oL-5bT2sbrObRZETpigUA2UtZ_hb9_7GPRbq5hRi7e5BBNAKEVRdgeaEhAU8TqO7vFsY2dVYm6wT0ccGhQ7g9Hz2dD4Ka-umaVuCzY1AtqiQw", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.32106189999999, 29.95185230000001], "type": "Point"}, "properties": {"address": "4420 Greens Rd, Houston, TX 77032, United States", "id": "0fea99fe6edf1976f5651dcee53a4c65b4b30561", "job_title": "geologist", "name": "Weatherford Us LP", "photos": [], "place_id": "ChIJb4d8jKW2QIYR9DvsxTaZDig", "rating": 4, "reference": "CmRRAAAAE7KQiY2CboLMMOJl0CKP9xFhkOsLaAFQ59I1K7NR8cirR4Cdsuaaa0_Yl7vQ8O3Ch_27HyqW-fsputd8kA3pSWb9Mh2Oea9qcPsOZ5hqyJLKARnsi-mNaQBlkwXm5pdCEhCO9NkkWOgph-EAaM3OYpCZGhRKCzJ_W-Z3jY5RuYaNi-aLyQpSaw", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.64149499999999, 29.7802045], "type": "Point"}, "properties": {"address": "15995 N Barkers Landing Rd#275, Houston, TX 77079, United States", "id": "487c0f4fad0c96f92333e8445ffb7df4bbdd794a", "job_title": "geologist", "name": "Weatherford", "photos": [], "place_id": "ChIJgWVOPeHbQIYRD310QFCTldM", "rating": 4, "reference": "CmRSAAAAU8keogqpVAUHD7AWz0F9ITOARy1r6Ur0lOQDrylQBdt68ymc6GUAUa5DFRfX2dwioFwKMU5aW33yFD2IzXDb0u101gC_CF2r0q82l1EMdpffDlJ-_pduvMFE3N1W--dGEhCA2GDT0fYmCAXQm5bssb0QGhQrEWUn-rYUH6ZMhaeKV3jADFRm_w", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.46547729999999, 29.82209649999999], "type": "Point"}, "properties": {"address": "5151 Milwee St, Houston, TX 77092, United States", "id": "7c57cdc787545fb5cdaa2ec867964d2a4a03a214", "job_title": "geologist", "name": "Weatherford", "photos": [], "place_id": "ChIJf8uI6mTGQIYRT2uM5UPk9Rc", "rating": "NA", "reference": "CmRRAAAAp3nIDNOuuZsVDZUKZhANxArKQQbabBka0V4E2Kld6_UtXxN8qao36-BosQjkaIZr-i_NKpm-fvF03DPGXujYnHD9HQKpNJIKDz42-wdo2bPVN6OQa_RL6YmhOziPZt65EhDtLw7BtIwuFGwMjVqN_6N4GhQjnParM6cW-G317chu0lwMM-O-AA", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5687557, 29.8280322], "type": "Point"}, "properties": {"address": "3600 Brittmoore Rd, Houston, TX 77043, United States", "id": "292fb3ea5cda6a1e89be2a7e1edb634766ad609d", "job_title": "geologist", "name": "Weatherford", "photos": [], "place_id": "ChIJYxAfN8naQIYRx5yd_B1HBy4", "rating": 4, "reference": "CmRRAAAAj6kpiwfDhdpU4xO_jhItviA-VK-xsI76T8V3mOFCt6tVJO-zjPH1WOl-0xP1EGZ37flL3gl7oDhQm_9zM_YOhQLR5MROW5t43VxYarA8So2HW5O00mXoYidGsT17t-TdEhBWZza9es0FtzyXvQ9fA_UbGhR4raL7nfJPESqs_v5CZIg9PwJKJA", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3308448, 29.9467059], "type": "Point"}, "properties": {"address": "15710 John F Kennedy Blvd #700, Houston, TX 77032, United States", "id": "8c57b1d23f4bbd0979cfdc9220021c397e740316", "job_title": "geologist", "name": "Weatherford International", "photos": [], "place_id": "ChIJ8QvAT722QIYRFoA0kAAlp_w", "rating": "NA", "reference": "CmRSAAAAZXryF4e6y1BLozdnMdoVPv3zIWkeT8d1FjJaiZa0avqKA-qtVIwQd6rAI3YkmYBgPzxmi5Oq8KMDl1WO4rxHbXbiLy7qH3V5aWWfDDo6mWL95nkicFs838GmJAMgLIFxEhC8DITwAz4GIN3Dd_GH1_MKGhS4qpkH01Wi_iy9Qj8ourRmXRgQxg", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.49729029999999, 29.9313766], "type": "Point"}, "properties": {"address": "10655 Bammel North Houston Rd, Houston, TX 77086, United States", "id": "ac2948f51dcbeb1d0caac8d3e79696ff62bb54df", "job_title": "geologist", "name": "Weatherford", "photos": [], "place_id": "ChIJ97T_sb_OQIYR3VDC1vCO01I", "rating": "NA", "reference": "CmRRAAAAE76pqgyOJN_IpUjSVkeQyDMjtR0H4OZ7Wmu2OUrVP4HGaAQXiwPNUXa_0dnqdZMT6CmRkEkk5MlrTmecIO0rzlOPfgWh6sn54h8c0QjJuPhClTfM0vB-8CoM-Ol89O5pEhCpqaekiCydnLUiCV2dTdUqGhQ5r04kwX9tLifMzsG1PpCPHhnbvQ", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5036944, 29.9384892], "type": "Point"}, "properties": {"address": "5200 North Sam Houston Pkwy W Suite 500, Houston, TX 77086, United States", "id": "c35235ad15c043ae6d3fa786e635bdc00ba7e525", "job_title": "geologist", "name": "Weatherford Labs", "photos": [], "place_id": "ChIJbarkM5fOQIYRtdFDh1I4Ibw", "rating": 5, "reference": "CmRSAAAAH9Mukgmsk7StTBpws9GKPPQnIcUBh_fWcmvEIXmH4Ez23dfqYI1uMkv-e_2Rl1F2I_us3TTap7WenxG2IlBEoZG-78vqxGElQBdh7QJCxPC3RRUGhuCQQF9jAh1zUSfwEhDZ_X8htO81DKQAlNLlRGl7GhQ5-e-j6Sn_ESNyknwXycz4mSidUw", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.58365099999999, 29.87864769999999], "type": "Point"}, "properties": {"address": "11909 FM 529 Road, Houston, TX 77041, United States", "id": "34a35815f125719e685a25e281877fd0f3e707dd", "job_title": "geologist", "name": "Weatherford", "photos": [], "place_id": "ChIJN6mSK2jQQIYRkXRkecpsqOk", "rating": 5, "reference": "CmRSAAAA_RhQ4VNwdnRGWmgBoC5AGqailMLV3RPkolV7pHTbC_YFZ0qQ5gPlRg9DF73_qaO9RyuzG8wAAKDITHsmieRKlOe3uCQ0MqveAxJRjpHJO_rVV6X4Cik9Zel_SDjJGMULEhBQ1msakpPSzyl-HmjOU8cVGhSQVumb_UB_5fbd-0TmBg2d02LwYg", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.551149, 29.9230152], "type": "Point"}, "properties": {"address": "8845 Fallbrook Dr, Houston, TX 77064, United States", "id": "6012e90f06cdf2fc9891b68f8e37c07102fc4043", "job_title": "geologist", "name": "Weatherford Labs", "photos": [], "place_id": "ChIJhSfAKyPOQIYR3TfWmiitgdk", "rating": "NA", "reference": "CmRSAAAAz4i4qja7vMqqr-7MgXyLvNpjiQgisbf0VX7AuLiIxfeEuDweRx6Lsp4Sat03vZkClRUdpLmkIa0_lBf6hsV4VKZu7e1AYmNCQxUxc5zOU-EEteWYLzijiQ5sqJPctwIGEhC4rJcYFodYjPVQ08RQyfYJGhSF4MzUPFoCZmo3ZneDIvhkQVrpCA", "types": ["school", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.48866699999999, 29.9435503], "type": "Point"}, "properties": {"address": "4420 W Greens Rd, Houston, TX 77066, United States", "id": "a0777591ff05d81829fbccf25c638ff0202494cb", "job_title": "geologist", "name": "Weatherford Wellcat", "photos": [], "place_id": "ChIJ7aT18q_OQIYRQVu3YUEkW8E", "rating": 5, "reference": "CmRSAAAAxUu6tJCokXKCYSFkV6z-xNtg7LQJLtISQncmYKfJLKeQgCGsCZaRc-X_jZYwZXwbrIKFt0-s4ZZIYCWdWu-XFuovSqBvp0aeFQD68Niq5zvA4bCHD0fw3TNn84kXMSZyEhCWLNk4PXaZ_l52A4TUJAHSGhSPpd_MRHP_TFjXkgzre-_p3sBAQQ", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.66870100000001, 29.786041], "type": "Point"}, "properties": {"address": "16430 Park Ten Pl, Houston, TX 77084, United States", "id": "1b41b81f14c05bef0bfa7b4b15ac1b253556cb23", "job_title": "geologist", "name": "Weatherford Us LP", "photos": [], "place_id": "ChIJK2PrHBXZQIYR5_WIovzEWWQ", "rating": "NA", "reference": "CmRRAAAA-LE8ZNBEWga1g4nl6aewEOC63X4weBuzXZfBP_ncu868gRFBpMh2d9IFiI9Vke3g45BD5Rdi3yQWSCnQ23epZp9jF256dbnkJF6ithZmGx_Za1Wicx9KVnCa7b6sVu8jEhCuoUxauaL2MdKhydWMVWaQGhS_K_q73KRVOHW0PD_rglNEyFVa7g", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.0431184, 29.6908544], "type": "Point"}, "properties": {"address": "11931 Hwy 225, La Porte, TX 77571, United States", "id": "e655333b194bd940ac7bab961f2556d75d128a2d", "job_title": "geologist", "name": "Weatherford", "photos": [], "place_id": "ChIJRe9oNwpgP4YRQAHsNXMU0vA", "rating": "NA", "reference": "CmRSAAAA7k2EZYd-RdkoBqDHxlKFjmJo4o6xvy5_aUu-qOuggUhl0Z7xJM0C0Jz-x7wvzZZgqXY4ctp2miwkLt3uxFPcspjDJOsGV3WN_79AANShm9wMGbsAqOcKzrN44d92cBczEhDpu4kyU824d6vNeymeSSK_GhT7RsDHNde1QASe7fk8s_Tj2tg58A", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.2559629, 30.0674877], "type": "Point"}, "properties": {"address": "22001 Northpark Dr, Kingwood, TX 77339, United States", "id": "1cf42089a61995ee434212bf913c60fbc8b01cd3", "job_title": "geologist", "name": "Weatherford", "photos": [{"height": 3264, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/116976098361550939190/photos\">Texan country75</a>"], "photo_reference": "CoQBdwAAAIexoitk8M9w2hS6zfGzOieEq3nqIbGqepUN6StOZKPJRUOHB3_4GsbEyusC780j7DfeJP3rkd5cq9oTvNwYBvmmpghqdoNmOpv5Y4d59qPrtYdsFSm88kiwfi7ql-FpHtsCDXDuAMAczg6EuCwLjEqpjAVg3tV2myUicO8t0A1SEhBAPhKzFOzN-kkqIXv1UiDOGhRcnApF5LSWVTS3gIH-ZpQf4Bf8Hg", "width": 1840}], "place_id": "ChIJ9_DxATtNR4YR5x9qO1Lvfsg", "rating": 4.5, "reference": "CmRSAAAAYUzVg2CUcygjvph7qTJvlrlgVDHSrYQJs81mrB5McbVhk5p5VGdFcbeVqO3p_ffZa_6pL-Idsb1W1vawbMGH_lK1geh5q6xNoYq8gJPtL7lfyJ7sBcb6ZQnnXzIfDeAOEhBc7gPQpaYLI8wtmI9cCxjOGhSDGKjfu8IHqW04G-wYujXKZdXj5g", "types": ["general_contractor", "car_repair", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.35959799999999, 29.766005], "type": "Point"}, "properties": {"address": "1 Main St, Houston, TX 77002, United States", "id": "bbf863152c3e55a67956f9a69a36bfe8822bc16e", "job_title": "geologist", "name": "University of Houston-Downtown", "photos": [{"height": 2988, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/113673164458451850004/photos\">raul medina torres</a>"], "photo_reference": "CoQBdwAAADSJgR0e_422_TiGdlxtjt1rtQXsPJgAHo8Jtx27FkSby9v8F9wdZcgCZBjPMm-Hb881HUcn9UGHrK8RSu4d-FQ14ve8MxdLF3knCcQFr_o8inw-Pz_UC1za6jXJc0BBRHrsBYrYgdiZqfyfCJQ6oKqhc4L4xO0vcWEr1A-Yfxu8EhDubUsI6D6AzipQAdeJqdDvGhQ-M-x8NTuaXAdYYyuq78OwY8DOGg", "width": 5312}], "place_id": "ChIJ0_q_Zyy_QIYRVx8D5OgE30I", "rating": 3.8, "reference": "CmRRAAAAMOVz-tXms-BIcZRPpwhfAjOeO_iXrL2_W_j9zz4TETzKaUWx6OzBHzYOoypCHfrz9kOrf5p42eUylWVMCyuTe27dSJxdU1zkzrIKQigORKGx-xbRaONwhYbWXjeG0nRoEhCXsMXGuifB2Z_7TWMndTBPGhTlBxqXuSlm3kmhnbDKoF2sbZ35xg", "types": ["university", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.372164, 29.757501], "type": "Point"}, "properties": {"address": "333 Clay St, Houston, TX 77002, United States", "id": "795a3515ff023985c96d55d60c8eefcdc6c9ee0e", "job_title": "geologist", "name": "Jefferies", "photos": [], "place_id": "ChIJe81fkji_QIYRhzwdUhh-ygw", "rating": "NA", "reference": "CmRRAAAAc74n0EieSy54y0Clj7wfUW7a73AGlVaZwrWAMbnx0aB7NysWIV-12AVj-E6c2ODiAPHd-x08-DiTjYU0rRlv2e91i23_xzENf9_YmKJNoJlaPuONKjYBx3_o8L_eS_xcEhB7UZqFc-fWskDOunTUP-j6GhRpfk7Fc24rtkFWUtexid_ysgnQFg", "types": ["finance", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.36392040000001, 29.7558061], "type": "Point"}, "properties": {"address": "1001 Fannin St # 1500, Houston, TX 77002, United States", "id": "f9c309a151d083c3d995d8e59499778d8c495ccd", "job_title": "geologist", "name": "Oasis Petroleum", "photos": [], "place_id": "ChIJr9ZsYDu_QIYR4jOwJRL56Zk", "rating": 5, "reference": "CmRSAAAA350wzGI1FySOvQ6o6--8GKvko4OhSxOgxQPNj_PTQ55DFo3ltUB_UVlj635N7W4y8dycF0xdWyXRL_vhKIgh2HhudcWqvHY4solXjTtwnyFlbXGCVnPpSC5mGQDDxH3sEhD-7bxyOsyiO3fKDjyCd0QIGhTjUZsk3339pBXMAKn_z6rXWTo2XA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5015712, 29.7513106], "type": "Point"}, "properties": {"address": "1616 S Voss Rd #1000, Houston, TX 77057, United States", "id": "a626299a58965fab800792adff33c7961eae6309", "job_title": "geologist", "name": "Hart Energy (Oil & Gas Data Provider)", "photos": [], "place_id": "ChIJRzKfmJPDQIYRcAIeSuMCH3k", "rating": 3, "reference": "CmRRAAAAtP5aP6gYlDTy4aIe6wldihGrLMvrGsiSusxGYHHlI3seDYG_6V2PzRXUj7m7IRBFB6cgaSLQ75ODpRNDzu42X26OrF2gNcqS6bmd0Q2e7o5gbsZ1O0GOpPrLrZ4p03esEhBFAxauykFXbxf06JHe-ypBGhT_I_41KcLWmcgLyZLwF9g4tkqTfg", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4628465, 29.7338536], "type": "Point"}, "properties": {"address": "3050 Post Oak Blvd # 1640, Houston, TX 77056, United States", "id": "eb737b548075ce9fb5a7b7cd19dec9171f0ee698", "job_title": "geologist", "name": "Sinopec USA", "photos": [], "place_id": "ChIJS5y97nHBQIYRbb2dw19hKiM", "rating": 5, "reference": "CmRRAAAABXhns3hxbM46_NvUFHyyox1Aewhx3ILtMemu0idJoXTAF-1qDJz8eyffvyDuzeLSUEenbIOYHs6wicZhpIv8c75F1RqI3SOnb4Ocfhun2gK6m6sJVkqRTJkYe6rHMs9gEhCvkk4Z9D4dcq--frcl2Fd2GhQh8PVZqO_OO4KR0fOVmrsstj6RIA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.46350079999999, 29.73921199999999], "type": "Point"}, "properties": {"address": "5065 Westheimer Rd #1100, Houston, TX 77056, United States", "id": "5e9393654923a4ebf381bb69db381436c29a299f", "job_title": "geologist", "name": "Citi Private Bank", "photos": [{"height": 4030, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/117652267326695258138/photos\">Ramesh Potti</a>"], "photo_reference": "CoQBdwAAAONZPxaMIoYvhgc0JOgRoWD1JzCGGRguCvhKqTdSp85zEHK5DYLhAEEB3xVmkN5YOehh-iaV6g3kJ2sPBkV0V-azLok-9pkRK_Yl6jn69VN4NlSPlmXdXFur2AcbV1R4a2CbqOvLA-w1YKdIUJj9t2cI7fl2Vdg8DJbHO183y9y5EhAxZ_O2pPqBx10sWqNO_wZ-GhTYHkaYbz3azHI-wuIKmPKLR3Rcag", "width": 3024}], "place_id": "ChIJFc2ux27BQIYRk9iQ3cBYcVk", "rating": 2.3, "reference": "CmRRAAAAyoTZEenQGK9IzV2qe833wGrgbh0zLWQ4eVfVptBySgqXEile5RCoXUIwmVh19ZQGGR0JJFZj-4W2Na_VPaWWiXfCktfDIOwWO9nco7-krK4zLdYAzTYkvar43dYpfKmSEhC8QzNo92jBBEqM97HefQtkGhSZnUS76w6UQxzFYkSso3vzjywxug", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.47219989999999, 29.7496917], "type": "Point"}, "properties": {"address": "5555 San Felipe St, Houston, TX 77056, United States", "id": "23d714827e55b6e7eb16a5d356a0feb120360633", "job_title": "geologist", "name": "Marathon Oil Company", "photos": [{"height": 2988, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/114653147413229085324/photos\">Patrick Winham</a>"], "photo_reference": "CoQBdwAAADY2mXH1lW7qDL0JANPuj02re6t4Y8MqKSnrZZt-XNbA8xCdK9mX6h1qvP1gfZihrnDmhccg2NFxR4anntR_oYJ7I50jqd57rzpf_qev7LKCcvCtPNxuby7OWSr1H-tiBsI04AmkypJ3bM1rlO8QnsrJnrwrVrNp87283a9dcZdLEhDZuinjemQ0iQmNZYXwfwwnGhSJjZvtb03xyfic4cGfJas0gBPVtA", "width": 5312}], "place_id": "ChIJieWsJFzBQIYR4INe7XAsF-g", "rating": 4.6, "reference": "CmRSAAAAV3cswHCrAwhbPwtB9qF1qyRgfbSNdf7KQj9d5XZLhFTSD7-A8M1_pvv_k_sgnBqotw8Qah9yinpqOsothYAHBIgRA8Lgmvd4G28I4cIb2ozOaLC3elhUEz27QnVYxImEEhAaJYx51Bhh1WwGhNl4WLZ3GhR5FvIB37oo1OYqH5Y8gZwXqYrJEw", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3699286, 29.7699427], "type": "Point"}, "properties": {"address": "PO Box 3128, Houston, TX 77253, United States", "id": "18997b37423029430e68e14ec02f1e975aa3a596", "job_title": "geologist", "name": "Marathon Oil Co: Chernosky Alvis", "photos": [], "place_id": "ChIJ64XB-DS_QIYR3wBPmVSvPQ0", "rating": "NA", "reference": "CmRRAAAAWUAEMUOoqcOk1nSaWS60Mt8wOkfRjzsm3H70fFo2ZDiNy3AQQ4hbxj_CbnUGet6LHVOYtP9OYVdCqoWBTCnWRcDtwaKzvrzE1FYdCoJC8P-3AWAuZ-ppzpWRq3reszmLEhCtyNbWeLM3ITP4S_NSVYRCGhRDuyVT1G4gG9EM_28L68GUtgAN4w", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3911105, 29.9585878], "type": "Point"}, "properties": {"address": "1030 Regional Park Dr, Houston, TX 77060, United States", "id": "591b862adf844e6fdf249a003d16efa649ab42b1", "job_title": "geologist", "name": "New Tech Global", "photos": [{"height": 556, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/105073521963953436703/photos\">New Tech Global</a>"], "photo_reference": "CoQBdwAAAGPUYHv_ZCz8m4omlYnrK0JRgvUo0fxUxMSAjcwAUDG_S3NNJI47WXoHeHKJ2bxXcyWujZtY1nFFBd29e8CkdAL3IGeCbKJ4T1IlaYm7-wI8Y9Ry7AMvF24w142KycdT5rh2LSIbra93oe35Hkrc24iOQ3W2IvYFQi-GeVuziFRSEhCOwnNfg4eE2ZCN9qITQfKRGhQpz-e_0Gy6LEDRT4PcpkhtUV0ohA", "width": 1186}], "place_id": "ChIJRQuoTf61QIYRrrCBvZ7uOWs", "rating": 5, "reference": "CmRRAAAAkfKn0653AAq2C6hg3bFS1Z70wdB3u2fE5gLHlhcPbeWrs_DOEntaeM3PFPtRjC59QK0YvAzbn8vlmcFkStpXa-yA86L1nOfwUd4Mu782g5yYCzxaYvD9L-PiWYWa5HcFEhBGJSg3iQaJaE34Vh7Nf2tfGhSRjbsV5lld-N0MSoGkhBgBofWtFA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4659039, 29.8679352], "type": "Point"}, "properties": {"address": "4141 Victory Dr, Houston, TX 77088, United States", "id": "90b73098291a8e90c04f33a8c85477dfcd26379d", "job_title": "geologist", "name": "Lone Star College System", "photos": [], "place_id": "ChIJqaVfqaXIQIYRf8PTc49aphc", "rating": 3, "reference": "CmRRAAAAtCwhe0PKqWt1glW5_3JXD0tjwgLHkfxH1TUneThawhq9e5LS7wSE2jdujvgTza-uyqSzBe77uHnNkPiqHXHH2ZWBf_gSVoxA-wZL6kL-3_kgfHKiP5VD4bKVY8B1e3FDEhAHdVgCOnQeq12KhuA-wflxGhR1nb2f2QSZWGVSWfFn8IDezDTfxA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.37974109999999, 30.007182], "type": "Point"}, "properties": {"address": "2700 W W Thorne Dr, Houston, TX 77073, United States", "id": "50e8c3703dd49384cfecda9f52ac2537c832cf50", "job_title": "geologist", "name": "Lone Star College-North Harris", "photos": [{"height": 428, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/106036001858778134743/photos\">Lone Star College-North Harris</a>"], "photo_reference": "CoQBdwAAAL3BG4OngxKOkafHj4giftyjYQiXTRhg2ayjPlg2ubBk5tWmb7qPEecuscL8bH1oIvVP_ni5ByVeqFi78jW4l8312jgVZzmwraHR8Elk0RyoQA9VnXtUNilUVm66pFtSrOTnsmcoep2mqHGghv9ZOEW5dxp4-amoFBPKN7uh3lzNEhC_baluH4gRhR5Yo_nwe8pVGhTQak3ToUQZlsNlJ57q26_Ktc_Vpw", "width": 640}], "place_id": "ChIJFwwy8Xa1QIYRMJtJ9403tZg", "rating": 3, "reference": "CmRSAAAAPthq35X4iwi64ysBZOtfMKNRdyDJeyqViPyn3EzQaH_8ZKGQoNKbSLGlU3U9mpWqfJKt6s7S0jB0r2zp5S9qo9Q7W5LVHKFHnWkQHavZR-S2HQ2-hg3lvwlZWzR_hX9hEhDf8j1M3EOQBjHB2KAb4l13GhQsJG5a750VqxjzYbqv-qlVYK9PJQ", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.48802210000001, 30.1872936], "type": "Point"}, "properties": {"address": "5000 Research Forest Dr, The Woodlands, TX 77381, United States", "id": "c4fbb88cbef4056a453a4e050728b4b6b1f46539", "job_title": "geologist", "name": "Lone Star College System", "photos": [{"height": 3264, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/113764785255023487240/photos\">Victor Brazon</a>"], "photo_reference": "CoQBdwAAAN-IxcBLceBxyTzWM6KWLk15jS9_jGq9mBdx54BFWPKPWR6iBJmz73qipGUpkZZZI2V_kIc913DbG36-sFtIRbm7sB-hEpEAr-oXWvkjp1qMcq7z48tcX30Rmn4aZCtXU3hJs98XNhSsYn2IDmcuA6muVhECf51ycXVzOXXypssdEhBse2sDG6hp6XMS0zxtLcdTGhTQP7Fr9FR4kF7oSIJD4M4ZAaZC4g", "width": 2448}], "place_id": "ChIJz5LplNkwR4YRI8q2M1XhwqQ", "rating": 2.5, "reference": "CmRSAAAAlU1LhDvhqB4Q2yvgq7FPew0l8_rs3Mm7to8ZMtXvqNlCymMDc_Y2IvBBrOf02vST7nVh4UM-D5Up_IP_MQcytTc_MWw6FnPRrNxC7QRM6a52ajTfLj74P-Ps46oeXJT-EhDefvmK972PdZLsRk3GZAPIGhTOc5EcSHcq36g5Lfu2VrxdBo6kdw", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.6012956, 29.7881337], "type": "Point"}, "properties": {"address": "12141 Wickchester Ln #200, Houston, TX 77079, United States", "id": "3cdf19db1aed68ee6eb0cb60d82ec77206b1eb4d", "job_title": "geophysicist", "name": "Geotrace Technologies Inc", "photos": [], "place_id": "ChIJE5-yOvzbQIYRJdZ1RfRAxzc", "rating": "NA", "reference": "CmRRAAAAChaQHV1AdPTv4Km5z6XZZBX8lHApJoGardEx6Z_QtDQMrB47GGa_e3jsNGWowTgeigPIxnW_IsSSoK0Sv54NivLgj1-YXGgVFTVZCij9FnF-o02JONh0H4ZMnXnlBgSCEhBIq7pNr0lLQ3rUjWCIYZeSGhQcUhVwrUVweA660F_B_HC8KMxUPw", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5438387, 29.7776847], "type": "Point"}, "properties": {"address": "800 Gessner Rd #170, Houston, TX 77024, United States", "id": "91282085c6930276af6f7e2abf1ee0e0fc18432c", "job_title": "geophysicist", "name": "Murray Resources", "photos": [{"height": 683, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/106500950429449697681/photos\">Murray Resources</a>"], "photo_reference": "CoQBdwAAAImgt2YBeinBolRWpSmlrDzcMe0heRO7PsREVXkalDo2qon0OfYcJ_g6RhQbTNyGhPm-BdCDxyAKIV-Ob3wBnshV_z_YOHXPa2menY-Lx7Ra7uCi9dd_uW020-xy7AtSAI3Nwh2ET8wRpPuFV7GSx_X5qFfOoFxDKygD0e4bIEdtEhANilhLmBAKKjOhjQUt3g8tGhQHFFtZsg-o8oUe4AyQ7HADQvKhqA", "width": 1024}], "place_id": "ChIJRSYpxLzEQIYRsKFA9Ahm-Z0", "rating": 4.6, "reference": "CmRSAAAAxQTedU7_eyvjR1-YXlw05Ts8EteidP5DrDe4-7lqdXuy_P9M7tJMOekXht1agy4fyrS8a7mB94y4uA4cZK4so7y-rwhhX3ceHoON0Zq84xYDVpCpSmCcsKDkzOGvH_vdEhCazJXvSwizPpaGQN7dZs0_GhSGM0OhOxuuRuSx5nI4--lJdBV3Kg", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3599019, 29.7546294], "type": "Point"}, "properties": {"address": "Hess Tower, 1501 McKinney St, Houston, TX 77010, United States", "id": "04c70c4a3d18050ca437bbaedeca70bd9cb01927", "job_title": "geophysicist", "name": "Hess Corporation Exploration & Production", "photos": [{"height": 1454, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/111482964284093817946/photos\">Sau Hei Lam</a>"], "photo_reference": "CoQBdwAAAE5-HLy2zHttUE50SckJ8MCbOk7aZALo1QQ8rsj2bQmZCSNh2GMohFSbF5VnH-3AkEum8k102xqDi-ByX0xOQkFa1HfO36jJDQz-mHaqRf8PZoiTsmwglC5CKvhbn-pExmigO82nb1W1ScGSixUt_WkgNE42vGQa3EoiBvpztBNhEhATUCRvns_JfE2pveCL95UqGhTF6Bf7H2xVHE5JYVs1WONZDOdrqg", "width": 2161}], "place_id": "ChIJfy2iBDm_QIYRjgKEE3GFL9Y", "rating": 4.4, "reference": "CmRSAAAAFQ1b8Pj1CGhfZY4dVyuIrI-gUgcr8DY-PdIeoQeqnA3I6H0RFeDvcZE0PHqf56a_6ZsEQFbGXfVOlf54_bYhEkhjY9EGsoQuzUs1xLLk2vq7JMaXlAQqSdHqTbTRqR7cEhCOu14Ryaf_U-Bcteo9op7XGhRQGlBuj8AbGVHx5sp0x3ljXyZQTA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4574449, 29.6839704], "type": "Point"}, "properties": {"address": "9009 West Loop S, Houston, TX 77096, United States", "id": "2795a232d6e43ce11fa01f15cf2d6001916581de", "job_title": "geophysicist", "name": "Aramco Services Company", "photos": [], "place_id": "ChIJ4-bQE-fBQIYRMJKxqTjw_Wc", "rating": 3, "reference": "CmRRAAAAn_tYgY3I6UcgD-e-Cqa7hdUvz55YYwnm38mxet2qhOc49H5H_WDzhIDzt9MJZBOlZ65pNBMX7U4EzFNfdsnjElFNmCfnP0DNP5bNcXt9veZMMNgJDiQzlzS9Y1AFbVDHEhC8AvTqHNzEQ7ghXVllSsTJGhTdzYS2ipKFTsZ0eMswC0DN6Tlj0g", "types": ["finance", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4628465, 29.7338536], "type": "Point"}, "properties": {"address": "3050 Post Oak Blvd # 1640, Houston, TX 77056, United States", "id": "eb737b548075ce9fb5a7b7cd19dec9171f0ee698", "job_title": "geophysicist", "name": "Sinopec USA", "photos": [], "place_id": "ChIJS5y97nHBQIYRbb2dw19hKiM", "rating": 5, "reference": "CmRRAAAAkcYnQhM-gNwgvkq7Sv8AkEkq1LQ3fCzRILWRsFCq7dzBRs3BOKj-Q70V8QsA8eZMuDjYhyCTIbGbxFlRa_VK8SET61FSyd8VVSD6QJotS60xRvHjVeGXd_0hBvRkNzBXEhD4-FwyjbHHOJ_GqBy3iGkiGhRlcHuaodhQkboq5bvv8zXkIeNf1Q", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5740797, 29.7373078], "type": "Point"}, "properties": {"address": "11200 Westheimer Rd #365, Houston, TX 77042, United States", "id": "328101c9c23ad52a56a549ad72e19915b58d7af8", "job_title": "geophysicist", "name": "Walker Elliott", "photos": [], "place_id": "ChIJz6iXr8HcQIYRnzHDA7T8m-k", "rating": 3, "reference": "CmRSAAAASQdbnuXOcsbi7HNR6fGZyjdXAtX1RId1MPZ8YTIo8xvfzUhrbtJ7Ck9UjhMSZm1m-h9S0K1trJXKykT7-ng-wSySVdhcllJsk3TC_nhrm8mGuWEUqfNke5_wRzYokUJ3EhBtHFToRupL1DiCTPW0vAeBGhRne8o8I8n9fcwKFUFD_rehKF8oQQ", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3662701, 29.7561894], "type": "Point"}, "properties": {"address": "1111 Travis St, Houston, TX 77002, United States", "id": "6cf1f2b19f724b7014c7cab1c01725834dbc3db4", "job_title": "geophysicist", "name": "Hilcorp Energy Co", "photos": [{"height": 2448, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/100318015046809414765/photos\">Ronald Payne</a>"], "photo_reference": "CoQBdwAAAJM5cVOcQp0Kb3rfpnCEs6z-jLYsN248MKcrETKt6KO4QMT_qg7HRd3LspjapAEoZRuW1Tg0cljQnA5TTrHMk-Y1oDkkEQI703p4scDbNrCJYLdrM5gsIbAZIiyBSNaYZqzmX9yJSnjG6felhj5d1CKG5b9VwylRxUNbne8izQAAEhC3TNrrwU-q7VxrYRO-pbs6GhSOETAyUn_BSeHDbTBaKmUSA4q26g", "width": 3264}], "place_id": "ChIJncGsRDm_QIYR0e57PbvugTc", "rating": 4, "reference": "CmRRAAAAbXcyb5TAJfk9l2O99SwgBGSAn6fPr2Jcnx8b4GrxaSghkFIEG41mrYeTblUwgsIsE7s4jbY65FEn_oj9m26me2n1IzbfG5vbFgNi2WrWoV_t09mJ6Rm8GrQpts7rqkQVEhCA-0OabtSChjx7I6oh2TpcGhRCcgh4kVYgByjO4g7jB9_IQflPwA", "types": ["general_contractor", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4321022, 29.7305064], "type": "Point"}, "properties": {"address": "5 Greenway Plaza Suite 110, Houston, TX 77046, United States", "id": "7394562c1f1a87dc4b6fa017b44a961df160f5ef", "job_title": "geophysicist", "name": "Occidental Petroleum Corp", "photos": [], "place_id": "ChIJjd5k24G8woAREjjHjBsq0Yk", "rating": 4, "reference": "CmRSAAAAeO8MlWjtXfgudEfdIFtcNcseM3hQ3gAG-FXEUfCzlragR-0dmtRoBJpKVT_0INhPmXDLZBDIuOu278vYhVtGy8kjCKFuGeIHtrqfUtx02IeRF5SeST6-139uIM5GJOosEhAnZZNz1YMGss-KQF1jWbO3GhQOacccCqq6TIKb6WhUFExEcCuEWg", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3970411, 29.9417677], "type": "Point"}, "properties": {"address": "363 North Sam Houston Pkwy E, Houston, TX 77060, United States", "id": "bcfc0554855f942f730466fd447878511b6aa0b4", "job_title": "geophysicist", "name": "Occidental Petroleum Corporation", "photos": [], "place_id": "ChIJx73rqePJQIYRPjss3jFElZo", "rating": "NA", "reference": "CmRSAAAASs2o2jmOViF1UKai7eRaqF7Ao2Sx755gwAN2tozs6HskmyKcJVBK8kt-cP8luiVz1QAyUM7pVUffdAWdjUiQo-aYAsNJvlqOeiBkrQaExP6gni4XB0o3gtsmQ2PNZl_bEhBfsMr48SIo7F05vxdXqOVkGhQZXBTwQxeTcq7cRsTIfNTkM-yehQ", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.5498597, 29.66888299999999], "type": "Point"}, "properties": {"address": "4301 Southwest Fwy, Houston, TX 77027, United States", "id": "6e40f1d16226178cc0cf90f68d007ada7cfe5617", "job_title": "geophysicist", "name": "Oxy USA Inc", "photos": [], "place_id": "ChIJj7uu3g7BQIYR1FyPVqiuar8", "rating": 1, "reference": "CmRSAAAAFVF8CDT9ZRdBB1wRM7wS3u3iqvjbVsDc2VRTwpCZ5w0Xl-P5FmThK_DUmUpyJgAuoecG2ojHvK31KIH-05v9aT5PeayZFVlbmFIP2-ulL5x_J0lhYix4qwDdgaa7B_TPEhDpVdeexXE-OhFuz3OgC6QjGhSZH-W_TEqXmC_tRmtqUeKGdxyptw", "types": ["gas_station", "car_repair", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.362117, 29.76005199999999], "type": "Point"}, "properties": {"address": "1001 Texas Ave #1020, Houston, TX 77002, United States", "id": "a11c06170d12ee3b5509eebe23f7a83e021bcdac", "job_title": "geophysicist", "name": "Lumina", "photos": [{"height": 808, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/104809654983163322864/photos\">Lumina</a>"], "photo_reference": "CoQBdwAAAHJ_vDmxVDLOITlR-esaqsd5odeGq71SVTc3ALw4kjXJTYeAlzqwficxoOloOUr43u7OaIofHrqe9XKiViVIf-79DJG_CUNa6_ghsNjtBAybM_30ies8aHehc4EY5Q1pfyTT2bnXYlB3SJ3szwWjRFe4yEok7trkBPjnhEg0ogtLEhCBdUojx9qiDbk-Rj5jgmFQGhTF4NfsQL4TZWLv89hwehx6VrkOtA", "width": 1024}], "place_id": "ChIJiUNxO0C_QIYRfHaVr4rYzPY", "rating": "NA", "reference": "CmRSAAAA0teO86yzS3LertZN4YF80vBPAucaBPHixY_jwzLwQau8YVJ6dHHrsmuMzXVicKARRlAR1vHJ6SUqzvapexUlb2anuNUMkfGcTV4UixJ6jD-u86gw0SlZKUQebQPA9N33EhBfI6WIYWv2E7cuIVRaRX6oGhQlZ1m8AhrAVPho_dxpiC0Bp3xiMA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.451906, 29.754584], "type": "Point"}, "properties": {"address": "50 Briar Hollow Ln, Houston, TX 77027, United States", "id": "7e34dad551ed799d5f99931250df058f8447acc9", "job_title": "geophysicist", "name": "Integrated Geophysics", "photos": [{"height": 1360, "html_attributions": ["<a href=\"https://maps.google.com/maps/contrib/118046956100093733952/photos\">Integrated Geophysics</a>"], "photo_reference": "CoQBdwAAAP0BXCJblHnpXV9MxVztwT8bGO0Zv85UexyMYUlamP63ASNX5RVP6M7KTe0aW8XUbz_oRFK3nJS8lAQuDCz2Oiu44MjIDNygcQqN7iidFleoN2B6fGvjx2ThDcDU7oEeWZ4GohxO48vRQF_rrXLi2wvZX5zu8_rxxQ4PALCmlgGQEhDU3WCuQMIs6XKN4EBeaxOxGhQxLV_EVeMyuKVfK_7nqV6lBYvUKg", "width": 1359}], "place_id": "ChIJvbQlyTvBQIYRm0AeqK4g8R8", "rating": "NA", "reference": "CmRRAAAAnS6AraEsMe86ms17YRPD6TRrNiFLreHbP0kJDhLByYQfUJV7RfkKeWK_w6G_hFkKzSxocL4Ye3s1rWeNvo1x0YVRnirSQ6tyuONcfv44a3VdQ01hmAJGsZGyfLNV7CiaEhAFZ8vCBzi0PBhUnZNtNjP6GhQiS6tlkJbqbvzEQIPz4Pmdq8ePkA", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.4628465, 29.7338536], "type": "Point"}, "properties": {"address": "3050 Post Oak Blvd # 1640, Houston, TX 77056, United States", "id": "eb737b548075ce9fb5a7b7cd19dec9171f0ee698", "job_title": "geophysicist", "name": "Sinopec USA", "photos": [], "place_id": "ChIJS5y97nHBQIYRbb2dw19hKiM", "rating": 5, "reference": "CmRRAAAAKR1qf1CSfOUmYe5hM3ayoOTrl8-rKAJXLOReULFSVK785_7hHgcA8z0HHvosCEMlKFAEZnbvgZvGAKx0AnCWvfyQiIF1TMNApQkGSEW-krlo4BU5tX61KXLVUwXv0HXiEhAgMr8L416_kyhqFzM9NvaiGhS-HWghz0UROsZ2RWjoB6R5JxUsag", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.62566849999999, 29.7794433], "type": "Point"}, "properties": {"address": "15375 Memorial Dr, Houston, TX 77079, United States", "id": "63e8846758150bbb7b96bb6f1b6575612bdc5048", "job_title": "geophysicist", "name": "PGS", "photos": [], "place_id": "ChIJ_Yxnp-zbQIYRwzVN-oronfk", "rating": "NA", "reference": "CmRSAAAART_c1MGvULr9wrcFbae-b0gMrLDkJ2h0l4i24nFTm8HocR2vDlBqoQSYy1c5cyo17OgkJp_s7_xBmatujTkZ3hfq431c5-0GYLojMbh8SccHHHz4E5T7BCCuSt0-3FJLEhBiiKs6Lr1Ot6xLyX2cXousGhQfYeF3i7opKe09iGelLOB3y-st4Q", "types": ["point_of_interest", "establishment"]}, "type": "Feature"}]

In [275]:
copied_feature_Collection = FeatureCollection(copied_feature_list)

{"features": [{"geometry": {"coordinates": [-95.46103649999999, 29.737203], "type": "Point"}, "properties": {"address": "2800 Post Oak Blvd #3900, Houston, TX 77056, United States", "id": "4b62e79323dd04edd16f18a3570c6c525c72352a", "job_title": "geologist", "name": "RBC Capital Markets", "photos": [], "place_id": "ChIJMzyxy23BQIYRmQh7_lj7cok", "rating": "NA", "reference": "CmRSAAAAZHCxUrcoNdmNHk-acO8JGRqvdpVGfoaJwc8gocBrMOwPY_kNrdG_voVRQhyobLytDJDE3z4no1B-jG-S1UU-XT65b8VH-a4MgXx5-b2DqbRErnd959mCH-Gbr5Bu7ptNEhA-to4Cld2aG366VDyB5m5RGhSDRAL2FObJKSL8ZwimNGfoWI8D1w", "types": ["bank", "finance", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3640089, 29.7604054], "type": "Point"}, "properties": {"address": "600 Travis St # 6500, Houston, TX 77002, United States", "id": "e6f1fffeaff13624b25a14bca2c17e581e517245", "job_title": "geologist", "name": "Royal Bank of Scotland", "photos": [], "place_id": "ChIJa0oElDq_QIYRxrrhsSUehHU", "rating": "NA", "

In [274]:
convertFeatureCollectionToGeoJSON(copied_feature_Collection,"../DATA/test_geojson_Everything_byHand_v2.geojson")

got to the function convertFeatureCollectionToGeoJSON


In [233]:
#### Two user supplied arguments
###### company_list_formatted = company_array_formatted
###### path = '../DATA/test_geojson_gAndg_v5'

make_geojson_from_CompanyJobList(company_list_formatted = company_array_formatted,path="../DATA/test_geojson_gAndg_v6.geojson",geographic_limits = geographic_limits)


job_title put into googleAPI call is = geologist
array_of_features =  [{"geometry": {"coordinates": [-95.46103649999999, 29.737203], "type": "Point"}, "properties": {"address": "2800 Post Oak Blvd #3900, Houston, TX 77056, United States", "id": "4b62e79323dd04edd16f18a3570c6c525c72352a", "job_title": "geologist", "name": "RBC Capital Markets", "photos": [], "place_id": "ChIJMzyxy23BQIYRmQh7_lj7cok", "rating": "NA", "reference": "CmRSAAAADuTV9QxW4KU1_EKpjj5ebTgzwkdDUESQitwdpp9YfgfYYRUSmFyfRPEeoZwSqmgjWzdxPFYGiUL6k5e-0m4zWCGeqij11FgFZ4f_lLekIL-UlGRNHi4xlScsOvL7-oAxEhAk9wNIaYp8Mmli3utI9Rc-GhRpp43ZIhrplKcv3GRQqrNQuMjKiA", "types": ["bank", "finance", "point_of_interest", "establishment"]}, "type": "Feature"}, {"geometry": {"coordinates": [-95.3640089, 29.7604054], "type": "Point"}, "properties": {"address": "600 Travis St # 6500, Houston, TX 77002, United States", "id": "e6f1fffeaff13624b25a14bca2c17e581e517245", "job_title": "geologist", "name": "Royal Bank of Scotland", "photos": [], "pl